# Classification Tree Program

In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, adjusted_rand_score

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, criterion='entropy'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.tree = None                                                          
        self.feature_importances = None                                          

    def entropy(self, y):
        counts = np.bincount(y)                                                  
        probabilities = counts / len(y)                                          
        return -np.sum([p * np.log2(p) for p in probabilities if p > 0])         

    def gini(self, y):
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def information_gain(self, y, left_indices, right_indices):
        if self.criterion == 'entropy':                                          
            impurity_func = self.entropy
        elif self.criterion == 'gini':
            impurity_func = self.gini
        else:
            raise ValueError(f"Unknown criterion: {self.criterion}")

        parent_impurity = impurity_func(y)                                       
        left_impurity = impurity_func(y[left_indices])
        right_impurity = impurity_func(y[right_indices])

        n, n_left, n_right = len(y), len(left_indices), len(right_indices)
        weighted_impurity = (n_left / n) * left_impurity + (n_right / n) * right_impurity
        inf_gain = parent_impurity - weighted_impurity
        
        # print(f'Inf. gain "{self.criterion}": {inf_gain}')
        return inf_gain                                                          
    
    
    def custom_1(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                               

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            b = 1
            
            # Избегаем деления на ноль
            # if p_1 > 0:
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            # if p_2 > 0:
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    
    
    def custom_2(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                             

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = np.sqrt(p_l)
            
            
            # if p_1 > 0:
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            # if p_2 > 0:
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    

    def custom_3(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                              

        sum_total = 0
        epsilon = 1e-10 
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = np.sqrt(p_l*(1 - p_l))
            
            # eps. для стабильности вычислений
            denominator_1 = max(p_1 * b**2, epsilon)
            denominator_2 = max(p_2 * b**2, epsilon)
            
            sum_total += ((p_1l - p_1 * p_l)**2) / denominator_1
            sum_total += ((p_2l - p_2 * p_l)**2) / denominator_2

        return N * sum_total
    

    def custom_4(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                               

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = p_l
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2


        return N * sum_total
    
    
    def custom_5(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                              

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = p_l**2
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    

    def custom_6(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                              

        sum_total = 0
        epsilon = 1e-10
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = -np.log(max(p_l, epsilon))
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    

    def custom_7(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                               

        sum_total = 0
        epsilon = 1e-10
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = (-p_l)*np.log(max(p_l, epsilon))
            
            # denominator_1 = max(p_1 * b**2, epsilon)
            # denominator_2 = max(p_2 * b**2, epsilon)
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    
    
    def custom_8(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                               

        sum_total = 0
        epsilon = 1e-10
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = -(p_l**0.5) * np.log(max(p_l, epsilon))
            
            # denominator_1 = max(p_1 * b**2, epsilon)
            # denominator_2 = max(p_2 * b**2, epsilon)
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total    
    

    def most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]


    def find_best_split(self, X, y, num_features, y_oh=None):
        best_gain = -float('inf')                                                  
        best_split = None                                                          

        for feature_index in range(num_features):                                  
            
            feature_values = np.sort(X[:, feature_index])
            
            thresholds = (feature_values[:-1] + feature_values[1:]) / 2     
            
            for threshold in thresholds:                                          
                left_indices = np.where(X[:, feature_index] <= threshold)[0]      
                right_indices = np.where(X[:, feature_index] > threshold)[0]      

                if (len(left_indices) < self.min_samples_leaf or 
                    len(right_indices) < self.min_samples_leaf):
                    continue                                                      

                if self.criterion == 'custom_1':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_1 criterion")
                    gain = self.custom_1(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_2':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_2 criterion")
                    gain = self.custom_2(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_3':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_3 criterion")
                    gain = self.custom_3(y_oh, left_indices, right_indices)                    
                
                elif self.criterion == 'custom_4':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_4 criterion")
                    gain = self.custom_4(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_5':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_5 criterion")
                    gain = self.custom_5(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_6':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_6 criterion")
                    gain = self.custom_6(y_oh, left_indices, right_indices)    
                    
                elif self.criterion == 'custom_7':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_7 criterion")
                    gain = self.custom_7(y_oh, left_indices, right_indices)
                    
                elif self.criterion == 'custom_8':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_7 criterion")
                    gain = self.custom_8(y_oh, left_indices, right_indices)                      
                
                else:
                    gain = self.information_gain(y, left_indices, right_indices)  

                if gain > best_gain:                                               
                    best_gain = gain                                               
                    best_split = {
                        'feature_index': feature_index,
                        'threshold': threshold,
                        'left_indices': left_indices,
                        'right_indices': right_indices,
                        'gain': gain
                    }                                                              
        
        return best_split                                                          


    def fit(self, X, y, y_oh=None):
        num_features = X.shape[1]
        self.feature_importances = np.zeros(num_features)                          
        self.tree = self.grow_tree(X, y, y_oh, depth=0)

        
        total = self.feature_importances.sum()
        if total > 0:
            self.feature_importances /= total


    def grow_tree(self, X, y, y_oh, depth):
        num_samples, num_features = X.shape
        num_classes = len(set(y))

        if (depth == self.max_depth or 
            num_classes == 1 or 
            num_samples < self.min_samples_split):
            return self.most_common_label(y)

        if self.criterion.startswith('custom_'):
            best_split = self.find_best_split(X, y, num_features, y_oh)
        else:
            best_split = self.find_best_split(X, y, num_features)

        if best_split is None:
            return self.most_common_label(y)

        left_indices, right_indices = best_split['left_indices'], best_split['right_indices']
        
        
        if self.criterion == 'custom_1':
            gain = self.custom_1(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_2':
            gain = self.custom_2(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_3':
            gain = self.custom_3(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_4':
            gain = self.custom_4(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_5':
            gain = self.custom_5(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_6':
            gain = self.custom_6(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_7':
            gain = self.custom_7(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_8':
            gain = self.custom_8(y_oh, left_indices, right_indices)            
            
        else:
            gain = self.information_gain(y, left_indices, right_indices)

        self.feature_importances[best_split['feature_index']] += gain              

        left_subtree = self.grow_tree(X[left_indices], y[left_indices], 
                                    y_oh[left_indices] if y_oh is not None else None, 
                                    depth + 1)
        right_subtree = self.grow_tree(X[right_indices], y[right_indices], 
                                     y_oh[right_indices] if y_oh is not None else None, 
                                     depth + 1)

        return {
            'feature_index': best_split['feature_index'],
            'threshold': best_split['threshold'],
            'left': left_subtree,
            'right': right_subtree
        }


    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])


    def _traverse_tree(self, x, node):
        if isinstance(node, dict):
            if x[node['feature_index']] <= node['threshold']:
                return self._traverse_tree(x, node['left'])
            else:
                return self._traverse_tree(x, node['right'])

        return node                                                           

### 1 Experiment

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


def compare_metrics_train_test(max_depth, X, y, *, N=None, V=None, k=None, alpha=None, nmin=None, random_state=42):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

    encoder = OneHotEncoder(sparse_output=False)
    y_oh_train = encoder.fit_transform(y_train.reshape(-1,1))


    '''Custom_1'''
    custom_1 = DecisionTree(max_depth=max_depth, criterion='custom_1')
    custom_1.fit(X_train, y_train, y_oh_train)
    y_pred = custom_1.predict(X_test)
    accuracy_1, precision_1 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_1, f1_1 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_1 = adjusted_rand_score(y_test, y_pred)

    '''GINI'''
    gini = DecisionTree(max_depth=max_depth, criterion='gini')
    gini.fit(X_train, y_train)
    y_pred = gini.predict(X_test)
    accuracy_gini, precision_gini = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_gini, f1_gini = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_gini = adjusted_rand_score(y_test, y_pred)

    '''Sklearn_GINI'''
    sk_gini = DecisionTreeClassifier(max_depth=max_depth, criterion='gini')
    sk_gini.fit(X_train, y_train)
    y_pred = sk_gini.predict(X_test)
    accuracy_gini_sk, precision_gini_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_gini_sk, f1_gini_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_gini_sk = adjusted_rand_score(y_test, y_pred)

    '''Entropy'''
    entropy = DecisionTree(max_depth=max_depth, criterion='entropy')
    entropy.fit(X_train, y_train)
    y_pred = entropy.predict(X_test)
    accuracy_entropy, precision_entropy = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_entropy, f1_entropy = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_entropy = adjusted_rand_score(y_test, y_pred)

    '''Sklearn_Entropy'''
    sk_entropy = DecisionTreeClassifier(max_depth=max_depth, criterion='entropy')
    sk_entropy.fit(X_train, y_train)
    y_pred = sk_entropy.predict(X_test)
    accuracy_entropy_sk, precision_entropy_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_entropy_sk, f1_entropy_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_entropy_sk = adjusted_rand_score(y_test, y_pred)
    
    '''Custom_2'''
    custom_2 = DecisionTree(max_depth=max_depth, criterion='custom_2')
    custom_2.fit(X_train, y_train, y_oh_train)
    y_pred = custom_2.predict(X_test)
    accuracy_2, precision_2 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_2, f1_2 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_2 = adjusted_rand_score(y_test, y_pred)

    '''Custom_3'''
    custom_3 = DecisionTree(max_depth=max_depth, criterion='custom_3')
    custom_3.fit(X_train, y_train, y_oh_train)
    y_pred = custom_3.predict(X_test)
    accuracy_3, precision_3 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_3, f1_3 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_3 = adjusted_rand_score(y_test, y_pred)

    '''Custom_4'''
    custom_4 = DecisionTree(max_depth=max_depth, criterion='custom_4')
    custom_4.fit(X_train, y_train, y_oh_train)
    y_pred = custom_4.predict(X_test)
    accuracy_4, precision_4 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_4, f1_4 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_4 = adjusted_rand_score(y_test, y_pred)

    '''Custom_5'''
    custom_5 = DecisionTree(max_depth=max_depth, criterion='custom_5')
    custom_5.fit(X_train, y_train, y_oh_train)
    y_pred = custom_5.predict(X_test)
    accuracy_5, precision_5 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_5, f1_5 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_5 = adjusted_rand_score(y_test, y_pred)

    '''Custom_6'''
    custom_6 = DecisionTree(max_depth=max_depth, criterion='custom_6')
    custom_6.fit(X_train, y_train, y_oh_train)
    y_pred = custom_6.predict(X_test)
    accuracy_6, precision_6 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_6, f1_6 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_6 = adjusted_rand_score(y_test, y_pred)

    '''Custom_7'''
    custom_7 = DecisionTree(max_depth=max_depth, criterion='custom_7')
    custom_7.fit(X_train, y_train, y_oh_train)
    y_pred = custom_7.predict(X_test)
    accuracy_7, precision_7 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_7, f1_7 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_7 = adjusted_rand_score(y_test, y_pred)
    
    '''Custom_8'''
    custom_8 = DecisionTree(max_depth=max_depth, criterion='custom_8')
    custom_8.fit(X_train, y_train, y_oh_train)
    y_pred = custom_8.predict(X_test)
    accuracy_8, precision_8 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_8, f1_8 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_8 = adjusted_rand_score(y_test, y_pred)

    results = np.round([[accuracy_1, accuracy_gini, accuracy_gini_sk, accuracy_entropy, accuracy_entropy_sk, accuracy_2, accuracy_3, accuracy_4, accuracy_5, accuracy_6, accuracy_7, accuracy_8],
                    [precision_1, precision_gini, precision_gini_sk, precision_entropy, precision_entropy_sk, precision_2, precision_3, precision_4, precision_5, precision_6, precision_7, precision_8],
                    [recall_1, recall_gini, recall_gini_sk, recall_entropy, recall_entropy_sk, recall_2, recall_3, recall_4, recall_5, recall_6, recall_7, recall_8],
                    [f1_1, f1_gini, f1_gini_sk, f1_entropy, f1_entropy_sk, f1_2, f1_3, f1_4, f1_5, f1_6, f1_7, f1_8],
                    [ari_1, ari_gini, ari_gini_sk, ari_entropy, ari_entropy_sk, ari_2, ari_3, ari_4, ari_5, ari_6, ari_7, ari_8],],4)

    column = ['b = 1','gini','gini_sklearn', 'entropy', 'entropy_sklearn', 'b = p_l ^ 0.5', 'b = (p_l*(1 - p_l)) ^ 0.5', 'b = p_l', 'b = p_l ^ 2', 'b = log(p_l)', 'b = -p_l * log(p_l)', 'b = p_l^0.5 * log(p_l)']
    table = pd.DataFrame(data=results, columns=column, index=['Accuracy', 'Precision', 'Recall','F1 score','ARI'])
    
    print(f'\nN, V, k, alpha, nmin, max_depth = {N, V, k, alpha, nmin, max_depth}')

    return table

### Mean/std of 50 exps.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


def compare_metrics_train_test(max_depth, X, y, *, N=None, V=None, k=None, alpha=None, nmin=None):
    
    all_results = []
    
    for seed in range(1,51):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

        encoder = OneHotEncoder(sparse_output=False)
        y_oh_train = encoder.fit_transform(y_train.reshape(-1,1))


        '''Custom_1'''
        custom_1 = DecisionTree(max_depth=max_depth, criterion='custom_1')
        custom_1.fit(X_train, y_train, y_oh_train)
        y_pred = custom_1.predict(X_test)
        accuracy_1, precision_1 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_1, f1_1 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_1 = adjusted_rand_score(y_test, y_pred)

        # '''GINI'''
        # gini = DecisionTree(max_depth=max_depth, criterion='gini')
        # gini.fit(X_train, y_train)
        # y_pred = gini.predict(X_test)
        # accuracy_gini, precision_gini = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        # recall_gini, f1_gini = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        # ari_gini = adjusted_rand_score(y_test, y_pred)

        # '''Sklearn_GINI'''
        # sk_gini = DecisionTreeClassifier(max_depth=max_depth, criterion='gini')
        # sk_gini.fit(X_train, y_train)
        # y_pred = sk_gini.predict(X_test)
        # accuracy_gini_sk, precision_gini_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        # recall_gini_sk, f1_gini_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        # ari_gini_sk = adjusted_rand_score(y_test, y_pred)

        # '''Entropy'''
        # entropy = DecisionTree(max_depth=max_depth, criterion='entropy')
        # entropy.fit(X_train, y_train)
        # y_pred = entropy.predict(X_test)
        # accuracy_entropy, precision_entropy = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        # recall_entropy, f1_entropy = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        # ari_entropy = adjusted_rand_score(y_test, y_pred)

        '''Sklearn_Entropy'''
        sk_entropy = DecisionTreeClassifier(max_depth=max_depth, criterion='entropy')
        sk_entropy.fit(X_train, y_train)
        y_pred = sk_entropy.predict(X_test)
        accuracy_entropy_sk, precision_entropy_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_entropy_sk, f1_entropy_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_entropy_sk = adjusted_rand_score(y_test, y_pred)
        
        '''Custom_2'''
        custom_2 = DecisionTree(max_depth=max_depth, criterion='custom_2')
        custom_2.fit(X_train, y_train, y_oh_train)
        y_pred = custom_2.predict(X_test)
        accuracy_2, precision_2 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_2, f1_2 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_2 = adjusted_rand_score(y_test, y_pred)

        '''Custom_3'''
        custom_3 = DecisionTree(max_depth=max_depth, criterion='custom_3')
        custom_3.fit(X_train, y_train, y_oh_train)
        y_pred = custom_3.predict(X_test)
        accuracy_3, precision_3 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_3, f1_3 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_3 = adjusted_rand_score(y_test, y_pred)

        '''Custom_4'''
        custom_4 = DecisionTree(max_depth=max_depth, criterion='custom_4')
        custom_4.fit(X_train, y_train, y_oh_train)
        y_pred = custom_4.predict(X_test)
        accuracy_4, precision_4 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_4, f1_4 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_4 = adjusted_rand_score(y_test, y_pred)

        '''Custom_5'''
        custom_5 = DecisionTree(max_depth=max_depth, criterion='custom_5')
        custom_5.fit(X_train, y_train, y_oh_train)
        y_pred = custom_5.predict(X_test)
        accuracy_5, precision_5 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_5, f1_5 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_5 = adjusted_rand_score(y_test, y_pred)

        '''Custom_6'''
        custom_6 = DecisionTree(max_depth=max_depth, criterion='custom_6')
        custom_6.fit(X_train, y_train, y_oh_train)
        y_pred = custom_6.predict(X_test)
        accuracy_6, precision_6 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_6, f1_6 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_6 = adjusted_rand_score(y_test, y_pred)

        '''Custom_7'''
        custom_7 = DecisionTree(max_depth=max_depth, criterion='custom_7')
        custom_7.fit(X_train, y_train, y_oh_train)
        y_pred = custom_7.predict(X_test)
        accuracy_7, precision_7 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_7, f1_7 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_7 = adjusted_rand_score(y_test, y_pred)


        '''Custom_8'''
        custom_8 = DecisionTree(max_depth=max_depth, criterion='custom_8')
        custom_8.fit(X_train, y_train, y_oh_train)
        y_pred = custom_8.predict(X_test)
        accuracy_8, precision_8 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_8, f1_8 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_8 = adjusted_rand_score(y_test, y_pred)

        results = np.round([[accuracy_1, accuracy_entropy_sk, accuracy_2, accuracy_3, accuracy_4, accuracy_5, accuracy_6, accuracy_7, accuracy_8],
                        [precision_1, precision_entropy_sk, precision_2, precision_3, precision_4, precision_5, precision_6, precision_7, precision_8],
                        [recall_1, recall_entropy_sk, recall_2, recall_3, recall_4, recall_5, recall_6, recall_7, recall_8],
                        [f1_1, f1_entropy_sk, f1_2, f1_3, f1_4, f1_5, f1_6, f1_7, f1_8],
                        [ari_1, ari_entropy_sk, ari_2, ari_3, ari_4, ari_5, ari_6, ari_7, ari_8],],4)
        
        all_results.append(results)
        # print(f'Finished: {seed} iter.')
        
    print(f'\nN, V, k, alpha, nmin, max_depth = {N, V, k, alpha, nmin, max_depth}')
            
    all_results = np.array(all_results)  

    mean_results = np.round(np.mean(all_results, axis=0),4)
    std_results = np.round(np.std(all_results, axis=0),4)
    
    # Final table (Mean/std)
    columns = ['b = 1','entropy_sklearn', 'b = p_l ^ 0.5', 'b = (p_l*(1 - p_l)) ^ 0.5', 'b = p_l', 'b = p_l ^ 2', 'b = -log(p_l)', 'b = -p_l * log(p_l)', 'b = -p_l^0.5 * log(p_l)']
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 score', 'ARI']
    
    index_tuples = []
    for metric in metrics:
        index_tuples.append((metric, 'Mean'))
        index_tuples.append((metric, 'Std'))
    
    multi_index = pd.MultiIndex.from_tuples(index_tuples, names=['Metric', 'Statistic'])
    
    # Final table
    final_table_data = []
    for i in range(len(metrics)):
        final_table_data.append(mean_results[i])
        final_table_data.append(std_results[i])
    
    final_table = pd.DataFrame(final_table_data, 
                             columns=columns, 
                             index=multi_index)
    
    return final_table

# Experiments with Generated and Real World Datasets

## Generated Datasets

---
#### Data generator

Parameters:
- N: Total number of data points
- V: Number of dimensions/features
- k: Number of clusters
- alpha: Controls cluster center spread (centers are in [α-1, 1-α])
- nmin: Minimum points per cluster
- seed: Random seed for reproducibility
- sig_range: Tuple (min, max) for cluster standard deviations

Returns:
- Nk: Array of cluster sizes
- R: List of ranges for each cluster
- y: Cluster labels for each point
- X: Generated data (N x V array)
- cen: Cluster centers (k x V array)

In [3]:
def generdat(N, V, k, alpha, nmin, seed=None, sig_range=(0.05, 0.1)):
    if N < k * nmin:
        raise ValueError(f"N must be >= k * nmin. Got N={N}, k={k}, nmin={nmin}")
    if k < 1:
        raise ValueError("k must be at least 1")
    if alpha == 1:
        raise ValueError("alpha cannot be 1")

    if seed is not None:
        np.random.seed(seed)

    if k == 1:
        Nk = np.array([N])
    else:
        base_sizes = np.ones(k, dtype=int) * nmin
        remaining = N - k * nmin
        if remaining > 0:
            additional = np.random.multinomial(remaining, np.ones(k)/k)
            Nk = base_sizes + additional
        else:
            Nk = base_sizes

    # Cluster centers
    cen = (alpha - 1) + 2 * (1 - alpha) * np.random.rand(k, V)

    X = np.zeros((N, V))
    y = np.zeros(N, dtype=int)
    R = []
    
    sig_min, sig_max = sig_range
    start_idx = 0
    
    for k0 in range(k):
        nk = Nk[k0]
        end_idx = start_idx + nk
        
        # Range for the current cluster
        R.append(range(start_idx, end_idx))
        y[start_idx:end_idx] = k0 
        
        # Cluster data generation
        sig = sig_min + (sig_max - sig_min) * np.random.rand(V)
        X[start_idx:end_idx] = np.random.randn(nk, V) * sig + cen[k0, :]
        
        start_idx = end_idx

    return Nk, R, y, X, cen

In [6]:
N, V, k, alpha, nmin = 500, 3, 4, 0.5, 50
Nk, R, y, X, cen = generdat(N, V, k, alpha, nmin, seed=101)

compare_metrics_train_test(max_depth=3, X=X, y=y, N=N, V=V, k=k, alpha=alpha, nmin=nmin)


N, V, k, alpha, nmin, max_depth = (500, 3, 4, 0.5, 50, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9914           0.9902         0.9896   
          Std        0.0076           0.0096         0.0104   
Precision Mean       0.9918           0.9907         0.9902   
          Std        0.0072           0.0091         0.0092   
Recall    Mean       0.9914           0.9902         0.9896   
          Std        0.0076           0.0096         0.0104   
F1 score  Mean       0.9914           0.9903         0.9896   
          Std        0.0076           0.0096         0.0105   
ARI       Mean       0.9779           0.9749         0.9739   
          Std        0.0197           0.0246         0.0247   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9912   0.9888       0.9874   
          Std                           0.0077   0.0109       0.0118   
Precision Mean                          0.9916   0.9896       0.9884   
          Std                           0.0073   0.0095       0.0101   
Recall    Mean                          0.9912   0.9888       0.9874   
          Std                           0.0077   0.0109       0.0118   
F1 score  Mean                          0.9912   0.9888       0.9874   
          Std                           0.0077   0.0109       0.0118   
ARI       Mean                          0.9775   0.9723       0.9691   
          Std                           0.0198   0.0254       0.0270   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9898               0.9915   
          Std               0.0099               0.0074   
Precision Mean              0.9902               0.9919   
          Std               0.0093               0.0069   
Recall    Mean              0.9898               0.9915   
          Std               0.0099               0.0074   
F1 score  Mean              0.9898               0.9915   
          Std               0.0098               0.0074   
ARI       Mean              0.9740               0.9784   
          Std               0.0252               0.0188   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9912  
          Std                         0.0077  
Precision Mean                        0.9916  
          Std                         0.0073  
Recall    Mean                        0.9912  
          Std                         0.0077  
F1 score  Mean                        0.9912  
          Std                         0.0077  
ARI       Mean                        0.9775  
          Std                         0.0198

### Generated Dataset / Tree_depth = 3

In [7]:
tables = []

for cluster in [4,8,15]:
    for feature in [6,15]:
        for squeeze in [0.5, 0.85]:
            N, V, k, alpha, nmin = 2000, feature, cluster, squeeze, 50
            Nk, R, y, X, cen = generdat(N, V, k, alpha, nmin, seed=101)
            
            table = compare_metrics_train_test(max_depth=3, X=X, y=y, N=N, V=V, k=k, alpha=alpha, nmin=nmin)
            tables.append(table)


N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.5, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.85, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.5, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.85, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.5, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.85, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.5, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.85, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.5, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.85, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.5, 50, 3)

N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.85, 50, 3)


In [8]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.5, 50, 3)
tables[0]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9933           0.9915         0.9933   
          Std        0.0030           0.0038         0.0030   
Precision Mean       0.9934           0.9916         0.9934   
          Std        0.0029           0.0038         0.0029   
Recall    Mean       0.9933           0.9915         0.9933   
          Std        0.0030           0.0038         0.0030   
F1 score  Mean       0.9933           0.9915         0.9933   
          Std        0.0030           0.0038         0.0030   
ARI       Mean       0.9822           0.9775         0.9822   
          Std        0.0077           0.0099         0.0077   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9936   0.9934       0.9929   
          Std                           0.0031   0.0030       0.0033   
Precision Mean                          0.9937   0.9935       0.9930   
          Std                           0.0031   0.0030       0.0032   
Recall    Mean                          0.9936   0.9934       0.9929   
          Std                           0.0031   0.0030       0.0033   
F1 score  Mean                          0.9936   0.9934       0.9929   
          Std                           0.0031   0.0030       0.0033   
ARI       Mean                          0.9831   0.9824       0.9812   
          Std                           0.0081   0.0079       0.0086   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9935               0.9934   
          Std               0.0028               0.0029   
Precision Mean              0.9935               0.9935   
          Std               0.0028               0.0028   
Recall    Mean              0.9935               0.9934   
          Std               0.0028               0.0029   
F1 score  Mean              0.9935               0.9934   
          Std               0.0028               0.0029   
ARI       Mean              0.9827               0.9824   
          Std               0.0072               0.0076   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9936  
          Std                         0.0031  
Precision Mean                        0.9936  
          Std                         0.0031  
Recall    Mean                        0.9936  
          Std                         0.0031  
F1 score  Mean                        0.9936  
          Std                         0.0031  
ARI       Mean                        0.9830  
          Std                         0.0080

In [9]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.85, 50, 3)
tables[1]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7312           0.7194         0.7254   
          Std        0.0196           0.0216         0.0218   
Precision Mean       0.7408           0.7432         0.7400   
          Std        0.0184           0.0188         0.0188   
Recall    Mean       0.7312           0.7194         0.7254   
          Std        0.0196           0.0216         0.0218   
F1 score  Mean       0.7300           0.7205         0.7259   
          Std        0.0191           0.0215         0.0198   
ARI       Mean       0.4544           0.4245         0.4393   
          Std        0.0377           0.0383         0.0427   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7356   0.7245       0.7245   
          Std                           0.0182   0.0212       0.0183   
Precision Mean                          0.7440   0.7381       0.7388   
          Std                           0.0176   0.0203       0.0207   
Recall    Mean                          0.7356   0.7245       0.7245   
          Std                           0.0182   0.0212       0.0183   
F1 score  Mean                          0.7337   0.7246       0.7248   
          Std                           0.0179   0.0200       0.0182   
ARI       Mean                          0.4642   0.4383       0.4362   
          Std                           0.0332   0.0423       0.0376   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7186               0.7271   
          Std               0.0250               0.0223   
Precision Mean              0.7146               0.7386   
          Std               0.0342               0.0191   
Recall    Mean              0.7186               0.7271   
          Std               0.0250               0.0223   
F1 score  Mean              0.7048               0.7264   
          Std               0.0353               0.0206   
ARI       Mean              0.4635               0.4459   
          Std               0.0308               0.0436   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7334  
          Std                         0.0183  
Precision Mean                        0.7401  
          Std                         0.0178  
Recall    Mean                        0.7334  
          Std                         0.0183  
F1 score  Mean                        0.7300  
          Std                         0.0185  
ARI       Mean                        0.4629  
          Std                         0.0341

In [10]:
#N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.5, 50, 3)
tables[2]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0026   
Precision Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0026   
Recall    Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0026   
F1 score  Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0026   
ARI       Mean       0.9939           0.9985         0.9934   
          Std        0.0065           0.0027         0.0070   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9978   0.9974       0.9975   
          Std                           0.0024   0.0025       0.0024   
Precision Mean                          0.9978   0.9975       0.9975   
          Std                           0.0024   0.0025       0.0024   
Recall    Mean                          0.9978   0.9974       0.9975   
          Std                           0.0024   0.0025       0.0024   
F1 score  Mean                          0.9978   0.9974       0.9975   
          Std                           0.0024   0.0025       0.0024   
ARI       Mean                          0.9941   0.9933       0.9935   
          Std                           0.0064   0.0066       0.0064   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9982               0.9974   
          Std               0.0022               0.0026   
Precision Mean              0.9983               0.9974   
          Std               0.0022               0.0025   
Recall    Mean              0.9982               0.9974   
          Std               0.0022               0.0026   
F1 score  Mean              0.9982               0.9974   
          Std               0.0022               0.0026   
ARI       Mean              0.9954               0.9930   
          Std               0.0059               0.0068   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9978  
          Std                         0.0024  
Precision Mean                        0.9978  
          Std                         0.0024  
Recall    Mean                        0.9978  
          Std                         0.0024  
F1 score  Mean                        0.9978  
          Std                         0.0024  
ARI       Mean                        0.9941  
          Std                         0.0063

In [11]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.85, 50, 3)
tables[3]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9514           0.9515         0.9502   
          Std        0.0076           0.0068         0.0088   
Precision Mean       0.9525           0.9527         0.9514   
          Std        0.0074           0.0065         0.0085   
Recall    Mean       0.9514           0.9515         0.9502   
          Std        0.0076           0.0068         0.0088   
F1 score  Mean       0.9515           0.9515         0.9502   
          Std        0.0076           0.0068         0.0088   
ARI       Mean       0.8744           0.8746         0.8715   
          Std        0.0189           0.0168         0.0219   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9521   0.9490       0.9464   
          Std                           0.0079   0.0102       0.0147   
Precision Mean                          0.9532   0.9502       0.9477   
          Std                           0.0076   0.0101       0.0143   
Recall    Mean                          0.9521   0.9490       0.9464   
          Std                           0.0079   0.0102       0.0147   
F1 score  Mean                          0.9522   0.9490       0.9464   
          Std                           0.0079   0.0102       0.0147   
ARI       Mean                          0.8763   0.8686       0.8621   
          Std                           0.0196   0.0256       0.0367   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9428               0.9498   
          Std               0.0311               0.0085   
Precision Mean              0.9466               0.9512   
          Std               0.0167               0.0080   
Recall    Mean              0.9428               0.9498   
          Std               0.0311               0.0085   
F1 score  Mean              0.9416               0.9499   
          Std               0.0405               0.0084   
ARI       Mean              0.8582               0.8700   
          Std               0.0417               0.0212   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9509  
          Std                         0.0082  
Precision Mean                        0.9521  
          Std                         0.0077  
Recall    Mean                        0.9509  
          Std                         0.0082  
F1 score  Mean                        0.9510  
          Std                         0.0081  
ARI       Mean                        0.8726  
          Std                         0.0205

In [12]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.5, 50, 3)
tables[4]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6444           0.8639         0.6156   
          Std        0.0730           0.0158         0.0435   
Precision Mean       0.5973           0.8033         0.5711   
          Std        0.1088           0.0190         0.0917   
Recall    Mean       0.6444           0.8639         0.6156   
          Std        0.0730           0.0158         0.0435   
F1 score  Mean       0.5575           0.8236         0.5375   
          Std        0.0866           0.0188         0.0490   
ARI       Mean       0.6129           0.8466         0.5236   
          Std        0.0944           0.0181         0.0951   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8579   0.6025       0.5420   
          Std                           0.0334   0.0309       0.0616   
Precision Mean                          0.7949   0.5722       0.5622   
          Std                           0.0477   0.0833       0.0843   
Recall    Mean                          0.8579   0.6025       0.5420   
          Std                           0.0334   0.0309       0.0616   
F1 score  Mean                          0.8160   0.5317       0.4821   
          Std                           0.0434   0.0328       0.0596   
ARI       Mean                          0.8406   0.4729       0.3596   
          Std                           0.0316   0.0717       0.0916   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7398               0.6181   
          Std               0.0428               0.0435   
Precision Mean              0.6743               0.5868   
          Std               0.0644               0.1007   
Recall    Mean              0.7398               0.6181   
          Std               0.0428               0.0435   
F1 score  Mean              0.6775               0.5362   
          Std               0.0491               0.0495   
ARI       Mean              0.6552               0.5399   
          Std               0.0639               0.0910   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6470  
          Std                         0.0757  
Precision Mean                        0.6136  
          Std                         0.1117  
Recall    Mean                        0.6470  
          Std                         0.0757  
F1 score  Mean                        0.5600  
          Std                         0.0896  
ARI       Mean                        0.6124  
          Std                         0.0924

In [13]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.85, 50, 3)
tables[5]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.5988           0.5947         0.5896   
          Std        0.0216           0.0224         0.0247   
Precision Mean       0.5420           0.5402         0.5345   
          Std        0.0205           0.0252         0.0329   
Recall    Mean       0.5988           0.5947         0.5896   
          Std        0.0216           0.0224         0.0247   
F1 score  Mean       0.5571           0.5530         0.5444   
          Std        0.0226           0.0235         0.0299   
ARI       Mean       0.3718           0.3678         0.3640   
          Std        0.0266           0.0229         0.0270   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.5712   0.5791       0.5535   
          Std                           0.0357   0.0321       0.0316   
Precision Mean                          0.5063   0.5258       0.4918   
          Std                           0.0571   0.0398       0.0475   
Recall    Mean                          0.5712   0.5791       0.5535   
          Std                           0.0357   0.0321       0.0316   
F1 score  Mean                          0.5145   0.5330       0.4958   
          Std                           0.0521   0.0361       0.0424   
ARI       Mean                          0.3701   0.3535       0.3283   
          Std                           0.0240   0.0340       0.0328   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5055               0.5916   
          Std               0.0251               0.0218   
Precision Mean              0.4137               0.5386   
          Std               0.0567               0.0272   
Recall    Mean              0.5055               0.5916   
          Std               0.0251               0.0218   
F1 score  Mean              0.4099               0.5474   
          Std               0.0344               0.0252   
ARI       Mean              0.3697               0.3655   
          Std               0.0232               0.0255   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5970  
          Std                         0.0212  
Precision Mean                        0.5405  
          Std                         0.0192  
Recall    Mean                        0.5970  
          Std                         0.0212  
F1 score  Mean                        0.5550  
          Std                         0.0221  
ARI       Mean                        0.3706  
          Std                         0.0258

In [14]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.5, 50, 3)
tables[6]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6652           0.9949         0.6709   
          Std        0.0804           0.0037         0.0874   
Precision Mean       0.5585           0.9950         0.5851   
          Std        0.0861           0.0036         0.0833   
Recall    Mean       0.6652           0.9949         0.6709   
          Std        0.0804           0.0037         0.0874   
F1 score  Mean       0.5821           0.9949         0.5988   
          Std        0.0911           0.0037         0.0961   
ARI       Mean       0.5971           0.9885         0.5678   
          Std        0.1275           0.0084         0.1297   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6745   0.6762       0.6612   
          Std                           0.0712   0.0874       0.1001   
Precision Mean                          0.5540   0.5945       0.6018   
          Std                           0.0797   0.0811       0.0829   
Recall    Mean                          0.6745   0.6762       0.6612   
          Std                           0.0712   0.0874       0.1001   
F1 score  Mean                          0.5867   0.6068       0.5942   
          Std                           0.0813   0.0955       0.1076   
ARI       Mean                          0.6338   0.5719       0.5494   
          Std                           0.1158   0.1258       0.1364   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6432               0.6713   
          Std               0.0704               0.0873   
Precision Mean              0.5276               0.5821   
          Std               0.0833               0.0845   
Recall    Mean              0.6432               0.6713   
          Std               0.0704               0.0873   
F1 score  Mean              0.5575               0.5974   
          Std               0.0814               0.0964   
ARI       Mean              0.5572               0.5748   
          Std               0.1113               0.1309   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6630  
          Std                         0.0786  
Precision Mean                        0.5530  
          Std                         0.0851  
Recall    Mean                        0.6630  
          Std                         0.0786  
F1 score  Mean                        0.5781  
          Std                         0.0895  
ARI       Mean                        0.6011  
          Std                         0.1248

In [15]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.85, 50, 3)
tables[7]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6307           0.6737         0.6105   
          Std        0.0307           0.0221         0.0362   
Precision Mean       0.5987           0.6241         0.5828   
          Std        0.0377           0.0252         0.0452   
Recall    Mean       0.6307           0.6737         0.6105   
          Std        0.0307           0.0221         0.0362   
F1 score  Mean       0.5871           0.6395         0.5617   
          Std        0.0359           0.0235         0.0403   
ARI       Mean       0.4296           0.4653         0.4119   
          Std        0.0347           0.0316         0.0498   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6224   0.5946       0.5654   
          Std                           0.0423   0.0472       0.0534   
Precision Mean                          0.5591   0.5701       0.5566   
          Std                           0.0703   0.0636       0.0694   
Recall    Mean                          0.6224   0.5946       0.5654   
          Std                           0.0423   0.0472       0.0534   
F1 score  Mean                          0.5686   0.5437       0.5194   
          Std                           0.0572   0.0564       0.0645   
ARI       Mean                          0.4278   0.3920       0.3497   
          Std                           0.0404   0.0595       0.0632   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5919               0.6148   
          Std               0.0468               0.0397   
Precision Mean              0.5462               0.5872   
          Std               0.0979               0.0490   
Recall    Mean              0.5919               0.6148   
          Std               0.0468               0.0397   
F1 score  Mean              0.5148               0.5659   
          Std               0.0704               0.0454   
ARI       Mean              0.4266               0.4147   
          Std               0.0285               0.0521   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6399  
          Std                         0.0315  
Precision Mean                        0.6147  
          Std                         0.0428  
Recall    Mean                        0.6399  
          Std                         0.0315  
F1 score  Mean                        0.5971  
          Std                         0.0411  
ARI       Mean                        0.4317  
          Std                         0.0350

In [16]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.5, 50, 3)
tables[8]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.4570           0.5115         0.4700   
          Std        0.0305           0.0200         0.0305   
Precision Mean       0.2902           0.3076         0.3269   
          Std        0.0355           0.0208         0.0440   
Recall    Mean       0.4570           0.5115         0.4700   
          Std        0.0305           0.0200         0.0305   
F1 score  Mean       0.3322           0.3705         0.3575   
          Std        0.0312           0.0215         0.0393   
ARI       Mean       0.4315           0.5668         0.4075   
          Std        0.0388           0.0312         0.0345   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.4863   0.4624       0.4410   
          Std                           0.0383   0.0446       0.0559   
Precision Mean                          0.3022   0.3374       0.3429   
          Std                           0.0440   0.0448       0.0428   
Recall    Mean                          0.4863   0.4624       0.4410   
          Std                           0.0383   0.0446       0.0559   
F1 score  Mean                          0.3546   0.3555       0.3401   
          Std                           0.0437   0.0492       0.0542   
ARI       Mean                          0.4896   0.3801       0.3386   
          Std                           0.0311   0.0591       0.0799   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5077               0.4694   
          Std               0.0233               0.0309   
Precision Mean              0.3491               0.3254   
          Std               0.0244               0.0421   
Recall    Mean              0.5077               0.4694   
          Std               0.0233               0.0309   
F1 score  Mean              0.3908               0.3578   
          Std               0.0246               0.0389   
ARI       Mean              0.4900               0.4038   
          Std               0.0236               0.0374   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.4602  
          Std                         0.0350  
Precision Mean                        0.2913  
          Std                         0.0381  
Recall    Mean                        0.4602  
          Std                         0.0350  
F1 score  Mean                        0.3328  
          Std                         0.0370  
ARI       Mean                        0.4434  
          Std                         0.0389

In [17]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.85, 50, 3)
tables[9]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.3508           0.3444         0.3534   
          Std        0.0187           0.0193         0.0179   
Precision Mean       0.2074           0.1858         0.2107   
          Std        0.0178           0.0214         0.0174   
Recall    Mean       0.3508           0.3444         0.3534   
          Std        0.0187           0.0193         0.0179   
F1 score  Mean       0.2464           0.2328         0.2489   
          Std        0.0178           0.0193         0.0170   
ARI       Mean       0.2371           0.2331         0.2372   
          Std        0.0171           0.0179         0.0167   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.3390   0.3477       0.3379   
          Std                           0.0200   0.0185       0.0194   
Precision Mean                          0.1812   0.2083       0.2070   
          Std                           0.0243   0.0179       0.0187   
Recall    Mean                          0.3390   0.3477       0.3379   
          Std                           0.0200   0.0185       0.0194   
F1 score  Mean                          0.2260   0.2448       0.2385   
          Std                           0.0212   0.0177       0.0177   
ARI       Mean                          0.2301   0.2312       0.2116   
          Std                           0.0184   0.0168       0.0227   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.3381               0.3527   
          Std               0.0204               0.0180   
Precision Mean              0.1776               0.2098   
          Std               0.0213               0.0167   
Recall    Mean              0.3381               0.3527   
          Std               0.0204               0.0180   
F1 score  Mean              0.2223               0.2485   
          Std               0.0191               0.0173   
ARI       Mean              0.2339               0.2383   
          Std               0.0196               0.0176   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.3489  
          Std                         0.0188  
Precision Mean                        0.2036  
          Std                         0.0196  
Recall    Mean                        0.3489  
          Std                         0.0188  
F1 score  Mean                        0.2429  
          Std                         0.0193  
ARI       Mean                        0.2387  
          Std                         0.0173

In [18]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.5, 50, 3)
tables[10]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.3900           0.5082         0.4049   
          Std        0.0434           0.0171         0.0567   
Precision Mean       0.2950           0.2977         0.3126   
          Std        0.0406           0.0186         0.0537   
Recall    Mean       0.3900           0.5082         0.4049   
          Std        0.0434           0.0171         0.0567   
F1 score  Mean       0.3045           0.3651         0.3227   
          Std        0.0385           0.0189         0.0549   
ARI       Mean       0.2347           0.5639         0.2426   
          Std        0.0685           0.0135         0.0792   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.3906   0.3807       0.3215   
          Std                           0.0191   0.0708       0.0678   
Precision Mean                          0.2933   0.3083       0.3096   
          Std                           0.0233   0.0580       0.0512   
Recall    Mean                          0.3906   0.3807       0.3215   
          Std                           0.0191   0.0708       0.0678   
F1 score  Mean                          0.3066   0.2989       0.2570   
          Std                           0.0182   0.0698       0.0597   
ARI       Mean                          0.1957   0.2269       0.1505   
          Std                           0.0289   0.0818       0.0775   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.3948               0.4012   
          Std               0.0193               0.0592   
Precision Mean              0.3018               0.3086   
          Std               0.0278               0.0509   
Recall    Mean              0.3948               0.4012   
          Std               0.0193               0.0592   
F1 score  Mean              0.3117               0.3168   
          Std               0.0213               0.0571   
ARI       Mean              0.2166               0.2471   
          Std               0.0481               0.0817   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.3942  
          Std                         0.0438  
Precision Mean                        0.2944  
          Std                         0.0394  
Recall    Mean                        0.3942  
          Std                         0.0438  
F1 score  Mean                        0.3085  
          Std                         0.0380  
ARI       Mean                        0.2452  
          Std                         0.0738

In [19]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.85, 50, 3)
tables[11]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.3825           0.3972         0.3836   
          Std        0.0228           0.0250         0.0197   
Precision Mean       0.2710           0.2376         0.2768   
          Std        0.0255           0.0204         0.0268   
Recall    Mean       0.3825           0.3972         0.3836   
          Std        0.0228           0.0250         0.0197   
F1 score  Mean       0.2984           0.2883         0.3010   
          Std        0.0213           0.0222         0.0197   
ARI       Mean       0.2066           0.2612         0.2035   
          Std        0.0223           0.0216         0.0243   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.3289   0.3826       0.3801   
          Std                           0.0330   0.0202       0.0238   
Precision Mean                          0.1873   0.2769       0.2816   
          Std                           0.0365   0.0249       0.0280   
Recall    Mean                          0.3289   0.3826       0.3801   
          Std                           0.0330   0.0202       0.0238   
F1 score  Mean                          0.2183   0.3001       0.2991   
          Std                           0.0361   0.0192       0.0229   
ARI       Mean                          0.2198   0.2024       0.1957   
          Std                           0.0238   0.0236       0.0207   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.3549               0.3817   
          Std               0.0250               0.0193   
Precision Mean              0.2387               0.2744   
          Std               0.0234               0.0267   
Recall    Mean              0.3549               0.3817   
          Std               0.0250               0.0193   
F1 score  Mean              0.2575               0.2990   
          Std               0.0261               0.0195   
ARI       Mean              0.2281               0.2034   
          Std               0.0232               0.0233   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.3832  
          Std                         0.0227  
Precision Mean                        0.2722  
          Std                         0.0253  
Recall    Mean                        0.3832  
          Std                         0.0227  
F1 score  Mean                        0.2991  
          Std                         0.0208  
ARI       Mean                        0.2077  
          Std                         0.0255

### Generated Dataset / Tree_depth = 4

In [20]:
tables = []

for cluster in [4,8,15]:
    for feature in [6,15]:
        for squeeze in [0.5, 0.85]:
            N, V, k, alpha, nmin = 2000, feature, cluster, squeeze, 50
            Nk, R, y, X, cen = generdat(N, V, k, alpha, nmin, seed=101)
            
            table = compare_metrics_train_test(max_depth=4, X=X, y=y, N=N, V=V, k=k, alpha=alpha, nmin=nmin)
            tables.append(table)


N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.5, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.85, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.5, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.85, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.5, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.85, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.5, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.85, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.5, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.85, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.5, 50, 4)

N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.85, 50, 4)


In [21]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.5, 50, 4)
tables[0]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9932           0.9926         0.9931   
          Std        0.0038           0.0040         0.0037   
Precision Mean       0.9933           0.9927         0.9932   
          Std        0.0038           0.0040         0.0036   
Recall    Mean       0.9932           0.9926         0.9931   
          Std        0.0038           0.0040         0.0037   
F1 score  Mean       0.9932           0.9926         0.9931   
          Std        0.0038           0.0040         0.0037   
ARI       Mean       0.9821           0.9804         0.9817   
          Std        0.0100           0.0104         0.0095   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9933   0.9935       0.9935   
          Std                           0.0037   0.0033       0.0035   
Precision Mean                          0.9934   0.9936       0.9935   
          Std                           0.0036   0.0033       0.0034   
Recall    Mean                          0.9933   0.9935       0.9935   
          Std                           0.0037   0.0033       0.0035   
F1 score  Mean                          0.9933   0.9935       0.9935   
          Std                           0.0037   0.0033       0.0035   
ARI       Mean                          0.9823   0.9828       0.9827   
          Std                           0.0095   0.0087       0.0090   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9931               0.9933   
          Std               0.0034               0.0036   
Precision Mean              0.9931               0.9933   
          Std               0.0033               0.0036   
Recall    Mean              0.9931               0.9933   
          Std               0.0034               0.0036   
F1 score  Mean              0.9931               0.9933   
          Std               0.0034               0.0036   
ARI       Mean              0.9817               0.9821   
          Std               0.0086               0.0094   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9932  
          Std                         0.0037  
Precision Mean                        0.9932  
          Std                         0.0037  
Recall    Mean                        0.9932  
          Std                         0.0037  
F1 score  Mean                        0.9932  
          Std                         0.0037  
ARI       Mean                        0.9819  
          Std                         0.0096

In [22]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 4, 0.85, 50, 4)
tables[1]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7576           0.7451         0.7529   
          Std        0.0160           0.0166         0.0173   
Precision Mean       0.7628           0.7567         0.7616   
          Std        0.0175           0.0187         0.0196   
Recall    Mean       0.7576           0.7451         0.7529   
          Std        0.0160           0.0166         0.0173   
F1 score  Mean       0.7571           0.7429         0.7533   
          Std        0.0164           0.0181         0.0173   
ARI       Mean       0.4937           0.4753         0.4810   
          Std        0.0273           0.0331         0.0316   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7567   0.7518       0.7493   
          Std                           0.0161   0.0185       0.0187   
Precision Mean                          0.7608   0.7627       0.7617   
          Std                           0.0175   0.0207       0.0204   
Recall    Mean                          0.7567   0.7518       0.7493   
          Std                           0.0161   0.0185       0.0187   
F1 score  Mean                          0.7553   0.7529       0.7508   
          Std                           0.0163   0.0180       0.0183   
ARI       Mean                          0.4960   0.4769       0.4707   
          Std                           0.0286   0.0337       0.0356   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7446               0.7517   
          Std               0.0171               0.0161   
Precision Mean              0.7565               0.7565   
          Std               0.0201               0.0181   
Recall    Mean              0.7446               0.7517   
          Std               0.0171               0.0161   
F1 score  Mean              0.7448               0.7503   
          Std               0.0173               0.0172   
ARI       Mean              0.4798               0.4838   
          Std               0.0300               0.0287   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7529  
          Std                         0.0160  
Precision Mean                        0.7566  
          Std                         0.0170  
Recall    Mean                        0.7529  
          Std                         0.0160  
F1 score  Mean                        0.7507  
          Std                         0.0175  
ARI       Mean                        0.4894  
          Std                         0.0268

In [23]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.5, 50, 4)
tables[2]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0026   
Precision Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0025   
Recall    Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0026   
F1 score  Mean       0.9977           0.9994         0.9975   
          Std        0.0025           0.0010         0.0026   
ARI       Mean       0.9939           0.9985         0.9935   
          Std        0.0065           0.0027         0.0068   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9978   0.9975       0.9976   
          Std                           0.0024   0.0025       0.0024   
Precision Mean                          0.9978   0.9975       0.9976   
          Std                           0.0024   0.0024       0.0023   
Recall    Mean                          0.9978   0.9975       0.9976   
          Std                           0.0024   0.0025       0.0024   
F1 score  Mean                          0.9978   0.9975       0.9976   
          Std                           0.0024   0.0025       0.0024   
ARI       Mean                          0.9941   0.9935       0.9937   
          Std                           0.0064   0.0065       0.0063   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9984               0.9974   
          Std               0.0023               0.0026   
Precision Mean              0.9984               0.9974   
          Std               0.0022               0.0025   
Recall    Mean              0.9984               0.9974   
          Std               0.0023               0.0026   
F1 score  Mean              0.9984               0.9974   
          Std               0.0023               0.0026   
ARI       Mean              0.9957               0.9930   
          Std               0.0059               0.0068   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9978  
          Std                         0.0024  
Precision Mean                        0.9978  
          Std                         0.0024  
Recall    Mean                        0.9978  
          Std                         0.0024  
F1 score  Mean                        0.9978  
          Std                         0.0024  
ARI       Mean                        0.9941  
          Std                         0.0063

In [24]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 4, 0.85, 50, 4)
tables[3]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9640           0.9660         0.9627   
          Std        0.0084           0.0078         0.0087   
Precision Mean       0.9646           0.9666         0.9634   
          Std        0.0082           0.0076         0.0085   
Recall    Mean       0.9640           0.9660         0.9627   
          Std        0.0084           0.0078         0.0087   
F1 score  Mean       0.9640           0.9660         0.9628   
          Std        0.0083           0.0078         0.0087   
ARI       Mean       0.9061           0.9113         0.9029   
          Std        0.0212           0.0199         0.0221   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9655   0.9627       0.9618   
          Std                           0.0089   0.0089       0.0092   
Precision Mean                          0.9661   0.9633       0.9624   
          Std                           0.0087   0.0088       0.0091   
Recall    Mean                          0.9655   0.9627       0.9618   
          Std                           0.0089   0.0089       0.0092   
F1 score  Mean                          0.9656   0.9627       0.9618   
          Std                           0.0089   0.0089       0.0092   
ARI       Mean                          0.9101   0.9029       0.9006   
          Std                           0.0225   0.0229       0.0236   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9572               0.9608   
          Std               0.0087               0.0083   
Precision Mean              0.9582               0.9616   
          Std               0.0084               0.0083   
Recall    Mean              0.9572               0.9608   
          Std               0.0087               0.0083   
F1 score  Mean              0.9573               0.9609   
          Std               0.0087               0.0083   
ARI       Mean              0.8892               0.8980   
          Std               0.0219               0.0210   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9608  
          Std                         0.0085  
Precision Mean                        0.9615  
          Std                         0.0080  
Recall    Mean                        0.9608  
          Std                         0.0085  
F1 score  Mean                        0.9608  
          Std                         0.0084  
ARI       Mean                        0.8977  
          Std                         0.0218

In [25]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.5, 50, 4)
tables[4]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8782           0.9896         0.8020   
          Std        0.0676           0.0038         0.0723   
Precision Mean       0.8998           0.9899         0.8498   
          Std        0.0874           0.0036         0.0929   
Recall    Mean       0.8782           0.9896         0.8020   
          Std        0.0676           0.0038         0.0723   
F1 score  Mean       0.8486           0.9896         0.7598   
          Std        0.0876           0.0038         0.0856   
ARI       Mean       0.8556           0.9765         0.7482   
          Std        0.0661           0.0083         0.1088   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9888   0.7602       0.6780   
          Std                           0.0049   0.0556       0.0634   
Precision Mean                          0.9892   0.8239       0.7611   
          Std                           0.0047   0.0954       0.0963   
Recall    Mean                          0.9888   0.7602       0.6780   
          Std                           0.0049   0.0556       0.0634   
F1 score  Mean                          0.9888   0.7140       0.6300   
          Std                           0.0049   0.0624       0.0652   
ARI       Mean                          0.9745   0.6763       0.5446   
          Std                           0.0111   0.0981       0.1018   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8769               0.8166   
          Std               0.0423               0.0703   
Precision Mean              0.8395               0.8561   
          Std               0.0723               0.0945   
Recall    Mean              0.8769               0.8166   
          Std               0.0423               0.0703   
F1 score  Mean              0.8399               0.7748   
          Std               0.0548               0.0844   
ARI       Mean              0.8574               0.7719   
          Std               0.0450               0.1005   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8800  
          Std                         0.0699  
Precision Mean                        0.9052  
          Std                         0.0881  
Recall    Mean                        0.8800  
          Std                         0.0699  
F1 score  Mean                        0.8506  
          Std                         0.0905  
ARI       Mean                        0.8563  
          Std                         0.0688

In [26]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 8, 0.85, 50, 4)
tables[5]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6496           0.6434         0.6495   
          Std        0.0199           0.0191         0.0196   
Precision Mean       0.6600           0.6599         0.6619   
          Std        0.0218           0.0209         0.0223   
Recall    Mean       0.6496           0.6434         0.6495   
          Std        0.0199           0.0191         0.0196   
F1 score  Mean       0.6439           0.6388         0.6430   
          Std        0.0206           0.0194         0.0194   
ARI       Mean       0.3923           0.3846         0.3926   
          Std        0.0260           0.0246         0.0279   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6380   0.6453       0.6303   
          Std                           0.0309   0.0219       0.0262   
Precision Mean                          0.6444   0.6574       0.6458   
          Std                           0.0394   0.0209       0.0255   
Recall    Mean                          0.6380   0.6453       0.6303   
          Std                           0.0309   0.0219       0.0262   
F1 score  Mean                          0.6297   0.6399       0.6263   
          Std                           0.0384   0.0212       0.0269   
ARI       Mean                          0.3869   0.3879       0.3709   
          Std                           0.0260   0.0295       0.0330   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5874               0.6512   
          Std               0.0294               0.0191   
Precision Mean              0.5783               0.6626   
          Std               0.0551               0.0222   
Recall    Mean              0.5874               0.6512   
          Std               0.0294               0.0191   
F1 score  Mean              0.5548               0.6451   
          Std               0.0419               0.0201   
ARI       Mean              0.3856               0.3940   
          Std               0.0271               0.0259   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6510  
          Std                         0.0196  
Precision Mean                        0.6606  
          Std                         0.0202  
Recall    Mean                        0.6510  
          Std                         0.0196  
F1 score  Mean                        0.6446  
          Std                         0.0204  
ARI       Mean                        0.3945  
          Std                         0.0265

In [27]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.5, 50, 4)
tables[6]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8502           0.9970         0.8134   
          Std        0.0997           0.0030         0.0963   
Precision Mean       0.7971           0.9970         0.7778   
          Std        0.1217           0.0029         0.1169   
Recall    Mean       0.8502           0.9970         0.8134   
          Std        0.0997           0.0030         0.0963   
F1 score  Mean       0.8060           0.9970         0.7668   
          Std        0.1253           0.0030         0.1112   
ARI       Mean       0.8335           0.9931         0.7662   
          Std        0.1158           0.0068         0.1403   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8852   0.8119       0.7981   
          Std                           0.0938   0.0931       0.0970   
Precision Mean                          0.8347   0.7908       0.7905   
          Std                           0.1294   0.1205       0.1215   
Recall    Mean                          0.8852   0.8119       0.7981   
          Std                           0.0938   0.0931       0.0970   
F1 score  Mean                          0.8494   0.7660       0.7519   
          Std                           0.1221   0.1062       0.1083   
ARI       Mean                          0.8799   0.7640       0.7399   
          Std                           0.0975   0.1353       0.1436   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8253               0.8216   
          Std               0.0907               0.0989   
Precision Mean              0.7543               0.7897   
          Std               0.1255               0.1229   
Recall    Mean              0.8253               0.8216   
          Std               0.0907               0.0989   
F1 score  Mean              0.7757               0.7759   
          Std               0.1153               0.1171   
ARI       Mean              0.7923               0.7796   
          Std               0.1152               0.1396   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8556  
          Std                         0.0988  
Precision Mean                        0.8002  
          Std                         0.1222  
Recall    Mean                        0.8556  
          Std                         0.0988  
F1 score  Mean                        0.8116  
          Std                         0.1257  
ARI       Mean                        0.8439  
          Std                         0.1077

In [28]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 8, 0.85, 50, 4)
tables[7]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7662           0.7667         0.7462   
          Std        0.0198           0.0176         0.0369   
Precision Mean       0.7860           0.7861         0.7709   
          Std        0.0186           0.0174         0.0280   
Recall    Mean       0.7662           0.7667         0.7462   
          Std        0.0198           0.0176         0.0369   
F1 score  Mean       0.7680           0.7686         0.7481   
          Std        0.0193           0.0175         0.0381   
ARI       Mean       0.5399           0.5415         0.5150   
          Std        0.0340           0.0295         0.0487   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7284   0.7306       0.7039   
          Std                           0.0368   0.0398       0.0503   
Precision Mean                          0.7508   0.7569       0.7355   
          Std                           0.0424   0.0363       0.0529   
Recall    Mean                          0.7284   0.7306       0.7039   
          Std                           0.0368   0.0398       0.0503   
F1 score  Mean                          0.7205   0.7316       0.7015   
          Std                           0.0472   0.0448       0.0629   
ARI       Mean                          0.5065   0.4977       0.4616   
          Std                           0.0409   0.0515       0.0553   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6804               0.7598   
          Std               0.0300               0.0404   
Precision Mean              0.7126               0.7813   
          Std               0.0442               0.0324   
Recall    Mean              0.6804               0.7598   
          Std               0.0300               0.0404   
F1 score  Mean              0.6536               0.7607   
          Std               0.0366               0.0420   
ARI       Mean              0.4968               0.5352   
          Std               0.0331               0.0547   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7682  
          Std                         0.0208  
Precision Mean                        0.7848  
          Std                         0.0195  
Recall    Mean                        0.7682  
          Std                         0.0208  
F1 score  Mean                        0.7691  
          Std                         0.0203  
ARI       Mean                        0.5452  
          Std                         0.0361

In [29]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.5, 50, 4)
tables[8]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7271           0.9004         0.6941   
          Std        0.0601           0.0291         0.0388   
Precision Mean       0.6603           0.8595         0.6531   
          Std        0.0730           0.0413         0.0614   
Recall    Mean       0.7271           0.9004         0.6941   
          Std        0.0601           0.0291         0.0388   
F1 score  Mean       0.6605           0.8731         0.6272   
          Std        0.0698           0.0381         0.0460   
ARI       Mean       0.6716           0.8580         0.6239   
          Std        0.0648           0.0372         0.0518   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8026   0.6805       0.6535   
          Std                           0.0443   0.0532       0.0675   
Precision Mean                          0.7310   0.6545       0.6648   
          Std                           0.0698   0.0665       0.0666   
Recall    Mean                          0.8026   0.6805       0.6535   
          Std                           0.0443   0.0532       0.0675   
F1 score  Mean                          0.7479   0.6155       0.5935   
          Std                           0.0572   0.0557       0.0686   
ARI       Mean                          0.7599   0.5892       0.5153   
          Std                           0.0477   0.0831       0.1131   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7158               0.6920   
          Std               0.0603               0.0413   
Precision Mean              0.6242               0.6537   
          Std               0.0744               0.0640   
Recall    Mean              0.7158               0.6920   
          Std               0.0603               0.0413   
F1 score  Mean              0.6361               0.6221   
          Std               0.0782               0.0487   
ARI       Mean              0.7082               0.6270   
          Std               0.0466               0.0532   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7435  
          Std                         0.0585  
Precision Mean                        0.6718  
          Std                         0.0644  
Recall    Mean                        0.7435  
          Std                         0.0585  
F1 score  Mean                        0.6785  
          Std                         0.0667  
ARI       Mean                        0.6903  
          Std                         0.0672

In [30]:
# N, V, k, alpha, nmin, max_depth = (2000, 6, 15, 0.85, 50, 4)
tables[9]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.5151           0.4993         0.5116   
          Std        0.0196           0.0206         0.0190   
Precision Mean       0.4842           0.4565         0.4910   
          Std        0.0245           0.0267         0.0233   
Recall    Mean       0.5151           0.4993         0.5116   
          Std        0.0196           0.0206         0.0190   
F1 score  Mean       0.4812           0.4615         0.4803   
          Std        0.0203           0.0225         0.0184   
ARI       Mean       0.2918           0.2926         0.2852   
          Std        0.0212           0.0200         0.0208   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.4835   0.5052       0.4783   
          Std                           0.0258   0.0211       0.0335   
Precision Mean                          0.4274   0.4882       0.4671   
          Std                           0.0436   0.0249       0.0330   
Recall    Mean                          0.4835   0.5052       0.4783   
          Std                           0.0258   0.0211       0.0335   
F1 score  Mean                          0.4330   0.4740       0.4466   
          Std                           0.0335   0.0200       0.0335   
ARI       Mean                          0.2805   0.2804       0.2632   
          Std                           0.0230   0.0218       0.0272   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.4888               0.5140   
          Std               0.0295               0.0210   
Precision Mean              0.4284               0.4928   
          Std               0.0466               0.0241   
Recall    Mean              0.4888               0.5140   
          Std               0.0295               0.0210   
F1 score  Mean              0.4369               0.4825   
          Std               0.0377               0.0206   
ARI       Mean              0.2867               0.2868   
          Std               0.0255               0.0224   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5153  
          Std                         0.0198  
Precision Mean                        0.4825  
          Std                         0.0260  
Recall    Mean                        0.5153  
          Std                         0.0198  
F1 score  Mean                        0.4801  
          Std                         0.0220  
ARI       Mean                        0.2946  
          Std                         0.0204

In [31]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.5, 50, 4)
tables[10]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.5426           0.9023         0.5552   
          Std        0.0667           0.0142         0.0793   
Precision Mean       0.4791           0.8728         0.5090   
          Std        0.0699           0.0259         0.0748   
Recall    Mean       0.5426           0.9023         0.5552   
          Std        0.0667           0.0142         0.0793   
F1 score  Mean       0.4712           0.8794         0.4903   
          Std        0.0690           0.0190         0.0804   
ARI       Mean       0.3928           0.8768         0.4045   
          Std        0.0963           0.0200         0.1129   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.5296   0.5362       0.4595   
          Std                           0.0321   0.0843       0.0891   
Precision Mean                          0.4541   0.5087       0.4975   
          Std                           0.0415   0.0761       0.0653   
Recall    Mean                          0.5296   0.5362       0.4595   
          Std                           0.0321   0.0843       0.0891   
F1 score  Mean                          0.4605   0.4699       0.4023   
          Std                           0.0375   0.0882       0.0832   
ARI       Mean                          0.3082   0.3656       0.2599   
          Std                           0.0599   0.1117       0.1135   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5228               0.5557   
          Std               0.0290               0.0797   
Precision Mean              0.4379               0.5129   
          Std               0.0425               0.0673   
Recall    Mean              0.5228               0.5557   
          Std               0.0290               0.0797   
F1 score  Mean              0.4476               0.4895   
          Std               0.0376               0.0812   
ARI       Mean              0.3338               0.4040   
          Std               0.0732               0.1177   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5494  
          Std                         0.0728  
Precision Mean                        0.4852  
          Std                         0.0692  
Recall    Mean                        0.5494  
          Std                         0.0728  
F1 score  Mean                        0.4779  
          Std                         0.0736  
ARI       Mean                        0.4120  
          Std                         0.1016

In [32]:
# N, V, k, alpha, nmin, max_depth = (2000, 15, 15, 0.85, 50, 4)
tables[11]

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.5150           0.5573         0.5042   
          Std        0.0254           0.0311         0.0249   
Precision Mean       0.5298           0.5278         0.5132   
          Std        0.0327           0.0430         0.0341   
Recall    Mean       0.5150           0.5573         0.5042   
          Std        0.0254           0.0311         0.0249   
F1 score  Mean       0.4900           0.5251         0.4778   
          Std        0.0266           0.0363         0.0260   
ARI       Mean       0.2821           0.3485         0.2793   
          Std        0.0308           0.0263         0.0304   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.4682   0.5032       0.4984   
          Std                           0.0479   0.0250       0.0268   
Precision Mean                          0.4113   0.5157       0.5089   
          Std                           0.0737   0.0360       0.0368   
Recall    Mean                          0.4682   0.5032       0.4984   
          Std                           0.0479   0.0250       0.0268   
F1 score  Mean                          0.4030   0.4759       0.4704   
          Std                           0.0592   0.0255       0.0257   
ARI       Mean                          0.2980   0.2772       0.2735   
          Std                           0.0364   0.0279       0.0288   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.4730               0.5088   
          Std               0.0384               0.0262   
Precision Mean              0.4343               0.5195   
          Std               0.0605               0.0369   
Recall    Mean              0.4730               0.5088   
          Std               0.0384               0.0262   
F1 score  Mean              0.4082               0.4795   
          Std               0.0507               0.0272   
ARI       Mean              0.3072               0.2819   
          Std               0.0318               0.0313   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5171  
          Std                         0.0289  
Precision Mean                        0.5364  
          Std                         0.0387  
Recall    Mean                        0.5171  
          Std                         0.0289  
F1 score  Mean                        0.4914  
          Std                         0.0290  
ARI       Mean                        0.2824  
          Std                         0.0346

## Real World Datasets (UCI Repository)

### Glass

In [36]:
df = pd.read_csv('../Datasets/glass.csv')
X = df.drop('Type', axis=1).to_numpy()
y = df['Type'].to_numpy()

In [37]:
compare_metrics_train_test(max_depth=3, X=X, y=y)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6533           0.6289         0.6437   
          Std        0.0512           0.0537         0.0635   
Precision Mean       0.6040           0.6307         0.6239   
          Std        0.0646           0.0651         0.0829   
Recall    Mean       0.6533           0.6289         0.6437   
          Std        0.0512           0.0537         0.0635   
F1 score  Mean       0.6134           0.6108         0.6180   
          Std        0.0517           0.0562         0.0677   
ARI       Mean       0.3154           0.2875         0.3111   
          Std        0.0749           0.0712         0.0863   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.5711   0.6148       0.6037   
          Std                           0.1033   0.0539       0.0477   
Precision Mean                          0.5195   0.6043       0.5913   
          Std                           0.1792   0.0746       0.0704   
Recall    Mean                          0.5711   0.6148       0.6037   
          Std                           0.1033   0.0539       0.0477   
F1 score  Mean                          0.5028   0.5903       0.5771   
          Std                           0.1435   0.0589       0.0513   
ARI       Mean                          0.3092   0.2808       0.2613   
          Std                           0.0725   0.0723       0.0753   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5607               0.6585   
          Std               0.1078               0.0581   
Precision Mean              0.4859               0.6239   
          Std               0.1897               0.0799   
Recall    Mean              0.5607               0.6585   
          Std               0.1078               0.0581   
F1 score  Mean              0.4844               0.6251   
          Std               0.1505               0.0634   
ARI       Mean              0.3060               0.3273   
          Std               0.0773               0.0809   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6596  
          Std                         0.0502  
Precision Mean                        0.6000  
          Std                         0.0687  
Recall    Mean                        0.6596  
          Std                         0.0502  
F1 score  Mean                        0.6148  
          Std                         0.0547  
ARI       Mean                        0.3302  
          Std                         0.0830

In [35]:
compare_metrics_train_test(max_depth=4, X=X, y=y)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6578           0.6626         0.6544   
          Std        0.0569           0.0585         0.0692   
Precision Mean       0.6515           0.6797         0.6649   
          Std        0.0790           0.0637         0.0777   
Recall    Mean       0.6578           0.6626         0.6544   
          Std        0.0569           0.0585         0.0692   
F1 score  Mean       0.6386           0.6531         0.6414   
          Std        0.0632           0.0608         0.0736   
ARI       Mean       0.3119           0.3168         0.3043   
          Std        0.0825           0.0790         0.0954   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6393   0.6315       0.6270   
          Std                           0.0763   0.0545       0.0567   
Precision Mean                          0.6286   0.6455       0.6504   
          Std                           0.1033   0.0577       0.0552   
Recall    Mean                          0.6393   0.6315       0.6270   
          Std                           0.0763   0.0545       0.0567   
F1 score  Mean                          0.6138   0.6208       0.6174   
          Std                           0.0926   0.0551       0.0533   
ARI       Mean                          0.3047   0.2794       0.2718   
          Std                           0.0763   0.0727       0.0674   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5907               0.6692   
          Std               0.1050               0.0576   
Precision Mean              0.5599               0.6654   
          Std               0.1680               0.0732   
Recall    Mean              0.5907               0.6692   
          Std               0.1050               0.0576   
F1 score  Mean              0.5373               0.6529   
          Std               0.1437               0.0623   
ARI       Mean              0.3000               0.3308   
          Std               0.0838               0.0816   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6652  
          Std                         0.0641  
Precision Mean                        0.6459  
          Std                         0.0842  
Recall    Mean                        0.6652  
          Std                         0.0641  
F1 score  Mean                        0.6425  
          Std                         0.0698  
ARI       Mean                        0.3285  
          Std                         0.0830

### Breast-cancer-prognostic

In [38]:
df = pd.read_csv('../DATASETS/breast_cancer_prognostic.csv')
df = df.drop('Unnamed: 0', axis=1)
df['Outcome'] = df['Outcome'].map({'R': 1, 'N': 0})
X = df.drop('Outcome', axis = 1).to_numpy()
y = df['Outcome'].to_numpy()

In [39]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6955           0.7098         0.6967   
          Std        0.0731           0.0801         0.0726   
Precision Mean       0.6558           0.6547         0.6565   
          Std        0.0874           0.0797         0.0887   
Recall    Mean       0.6955           0.7098         0.6967   
          Std        0.0731           0.0801         0.0726   
F1 score  Mean       0.6646           0.6670         0.6659   
          Std        0.0783           0.0732         0.0782   
ARI       Mean       0.0175           0.0089         0.0207   
          Std        0.0887           0.0607         0.0913   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6955   0.6959       0.6967   
          Std                           0.0723   0.0720       0.0720   
Precision Mean                          0.6561   0.6563       0.6572   
          Std                           0.0870   0.0869       0.0872   
Recall    Mean                          0.6955   0.6959       0.6967   
          Std                           0.0723   0.0720       0.0720   
F1 score  Mean                          0.6647   0.6650       0.6658   
          Std                           0.0778   0.0776       0.0779   
ARI       Mean                          0.0174   0.0176       0.0192   
          Std                           0.0888   0.0887       0.0869   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6947               0.6959   
          Std               0.0736               0.0708   
Precision Mean              0.6559               0.6562   
          Std               0.0874               0.0866   
Recall    Mean              0.6947               0.6959   
          Std               0.0736               0.0708   
F1 score  Mean              0.6644               0.6650   
          Std               0.0784               0.0770   
ARI       Mean              0.0177               0.0170   
          Std               0.0889               0.0883   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6963  
          Std                         0.0725  
Precision Mean                        0.6547  
          Std                         0.0873  
Recall    Mean                        0.6963  
          Std                         0.0725  
F1 score  Mean                        0.6648  
          Std                         0.0777  
ARI       Mean                        0.0177  
          Std                         0.0899

In [40]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6629           0.6939         0.6633   
          Std        0.0639           0.0777         0.0650   
Precision Mean       0.6516           0.6661         0.6528   
          Std        0.0800           0.0857         0.0789   
Recall    Mean       0.6629           0.6939         0.6633   
          Std        0.0639           0.0777         0.0650   
F1 score  Mean       0.6499           0.6661         0.6512   
          Std        0.0696           0.0796         0.0699   
ARI       Mean       0.0079           0.0266         0.0090   
          Std        0.0650           0.0821         0.0614   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6625   0.6620       0.6625   
          Std                           0.0657   0.0658       0.0652   
Precision Mean                          0.6517   0.6516       0.6520   
          Std                           0.0780   0.0780       0.0779   
Recall    Mean                          0.6625   0.6620       0.6625   
          Std                           0.0657   0.0658       0.0652   
F1 score  Mean                          0.6500   0.6497       0.6502   
          Std                           0.0698   0.0698       0.0695   
ARI       Mean                          0.0073   0.0073       0.0077   
          Std                           0.0621   0.0621       0.0621   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6600               0.6633   
          Std               0.0665               0.0634   
Precision Mean              0.6505               0.6511   
          Std               0.0783               0.0784   
Recall    Mean              0.6600               0.6633   
          Std               0.0665               0.0634   
F1 score  Mean              0.6483               0.6500   
          Std               0.0704               0.0690   
ARI       Mean              0.0061               0.0065   
          Std               0.0652               0.0623   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6608  
          Std                         0.0661  
Precision Mean                        0.6508  
          Std                         0.0782  
Recall    Mean                        0.6608  
          Std                         0.0661  
F1 score  Mean                        0.6489  
          Std                         0.0701  
ARI       Mean                        0.0064  
          Std                         0.0617

### Congressional voting records

In [41]:
df = pd.read_csv('../DATASETS/congressional-voting-records.csv')
X = df.drop('Class Name', axis=1).to_numpy()
y = df['Class Name'].to_numpy()

In [42]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9493           0.9514         0.9486   
          Std        0.0270           0.0269         0.0266   
Precision Mean       0.9524           0.9541         0.9517   
          Std        0.0241           0.0241         0.0237   
Recall    Mean       0.9493           0.9514         0.9486   
          Std        0.0270           0.0269         0.0266   
F1 score  Mean       0.9493           0.9513         0.9486   
          Std        0.0271           0.0270         0.0267   
ARI       Mean       0.8070           0.8146         0.8044   
          Std        0.0965           0.0961         0.0948   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9493   0.9493       0.9493   
          Std                           0.0270   0.0270       0.0270   
Precision Mean                          0.9524   0.9524       0.9524   
          Std                           0.0241   0.0241       0.0241   
Recall    Mean                          0.9493   0.9493       0.9493   
          Std                           0.0270   0.0270       0.0270   
F1 score  Mean                          0.9493   0.9493       0.9493   
          Std                           0.0271   0.0271       0.0271   
ARI       Mean                          0.8070   0.8070       0.8070   
          Std                           0.0965   0.0965       0.0965   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9493               0.9493   
          Std               0.0270               0.0270   
Precision Mean              0.9524               0.9524   
          Std               0.0241               0.0241   
Recall    Mean              0.9493               0.9493   
          Std               0.0270               0.0270   
F1 score  Mean              0.9493               0.9493   
          Std               0.0271               0.0271   
ARI       Mean              0.8070               0.8070   
          Std               0.0965               0.0965   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9493  
          Std                         0.0270  
Precision Mean                        0.9524  
          Std                         0.0241  
Recall    Mean                        0.9493  
          Std                         0.0270  
F1 score  Mean                        0.9493  
          Std                         0.0271  
ARI       Mean                        0.8070  
          Std                         0.0965

In [43]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.9483           0.9490         0.9493   
          Std        0.0272           0.0269         0.0277   
Precision Mean       0.9513           0.9520         0.9523   
          Std        0.0241           0.0231         0.0247   
Recall    Mean       0.9483           0.9490         0.9493   
          Std        0.0272           0.0269         0.0277   
F1 score  Mean       0.9483           0.9490         0.9493   
          Std        0.0272           0.0270         0.0277   
ARI       Mean       0.8033           0.8058         0.8072   
          Std        0.0969           0.0960         0.0990   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.9493   0.9486       0.9486   
          Std                           0.0272   0.0275       0.0275   
Precision Mean                          0.9521   0.9517       0.9517   
          Std                           0.0243   0.0244       0.0244   
Recall    Mean                          0.9493   0.9486       0.9486   
          Std                           0.0272   0.0275       0.0275   
F1 score  Mean                          0.9493   0.9486       0.9486   
          Std                           0.0273   0.0275       0.0275   
ARI       Mean                          0.8071   0.8046       0.8046   
          Std                           0.0975   0.0982       0.0982   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.9486               0.9483   
          Std               0.0275               0.0272   
Precision Mean              0.9517               0.9513   
          Std               0.0244               0.0241   
Recall    Mean              0.9486               0.9483   
          Std               0.0275               0.0272   
F1 score  Mean              0.9486               0.9483   
          Std               0.0275               0.0272   
ARI       Mean              0.8046               0.8033   
          Std               0.0982               0.0969   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.9483  
          Std                         0.0272  
Precision Mean                        0.9513  
          Std                         0.0241  
Recall    Mean                        0.9483  
          Std                         0.0272  
F1 score  Mean                        0.9483  
          Std                         0.0272  
ARI       Mean                        0.8033  
          Std                         0.0969

### Balance-scale

In [44]:
df = pd.read_csv('../Datasets/balance-scale-preprocessed.csv')
X = df.drop('Class', axis=1).to_numpy()
y = df['Class'].to_numpy()

In [45]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7092           0.6989         0.7055   
          Std        0.0326           0.0266         0.0319   
Precision Mean       0.6612           0.6511         0.6573   
          Std        0.0370           0.0316         0.0353   
Recall    Mean       0.7092           0.6989         0.7055   
          Std        0.0326           0.0266         0.0319   
F1 score  Mean       0.6811           0.6707         0.6774   
          Std        0.0344           0.0292         0.0338   
ARI       Mean       0.2427           0.2219         0.2361   
          Std        0.0606           0.0457         0.0581   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7085   0.7056       0.7046   
          Std                           0.0310   0.0314       0.0317   
Precision Mean                          0.6600   0.6571       0.6574   
          Std                           0.0355   0.0342       0.0351   
Recall    Mean                          0.7085   0.7056       0.7046   
          Std                           0.0310   0.0314       0.0317   
F1 score  Mean                          0.6804   0.6774       0.6764   
          Std                           0.0326   0.0331       0.0339   
ARI       Mean                          0.2414   0.2364       0.2339   
          Std                           0.0568   0.0581       0.0574   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7118               0.7108   
          Std               0.0304               0.0307   
Precision Mean              0.6651               0.6639   
          Std               0.0337               0.0335   
Recall    Mean              0.7118               0.7108   
          Std               0.0304               0.0307   
F1 score  Mean              0.6835               0.6825   
          Std               0.0316               0.0316   
ARI       Mean              0.2473               0.2454   
          Std               0.0600               0.0608   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7121  
          Std                         0.0351  
Precision Mean                        0.6648  
          Std                         0.0364  
Recall    Mean                        0.7121  
          Std                         0.0351  
F1 score  Mean                        0.6839  
          Std                         0.0352  
ARI       Mean                        0.2494  
          Std                         0.0713

In [46]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7825           0.7442         0.7767   
          Std        0.0320           0.0261         0.0334   
Precision Mean       0.7334           0.7067         0.7331   
          Std        0.0357           0.0326         0.0383   
Recall    Mean       0.7825           0.7442         0.7767   
          Std        0.0320           0.0261         0.0334   
F1 score  Mean       0.7551           0.7199         0.7502   
          Std        0.0332           0.0280         0.0342   
ARI       Mean       0.4173           0.3239         0.4031   
          Std        0.0757           0.0633         0.0773   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7782   0.7753       0.7720   
          Std                           0.0310   0.0308       0.0315   
Precision Mean                          0.7295   0.7302       0.7288   
          Std                           0.0328   0.0391       0.0395   
Recall    Mean                          0.7782   0.7753       0.7720   
          Std                           0.0310   0.0308       0.0315   
F1 score  Mean                          0.7508   0.7485       0.7454   
          Std                           0.0310   0.0324       0.0334   
ARI       Mean                          0.4063   0.3981       0.3900   
          Std                           0.0744   0.0726       0.0745   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7761               0.7833   
          Std               0.0276               0.0255   
Precision Mean              0.7277               0.7341   
          Std               0.0299               0.0316   
Recall    Mean              0.7761               0.7833   
          Std               0.0276               0.0255   
F1 score  Mean              0.7487               0.7558   
          Std               0.0277               0.0274   
ARI       Mean              0.4000               0.4176   
          Std               0.0672               0.0625   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7847  
          Std                         0.0279  
Precision Mean                        0.7341  
          Std                         0.0305  
Recall    Mean                        0.7847  
          Std                         0.0279  
F1 score  Mean                        0.7567  
          Std                         0.0283  
ARI       Mean                        0.4212  
          Std                         0.0697

### Blood-transfusion

In [47]:
df = pd.read_csv('../Datasets/blood-transfusion.csv')
X = df.drop('whether he/she donated blood in March 2007', axis=1).to_numpy()
y = df['whether he/she donated blood in March 2007'].to_numpy()

In [48]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7757           0.7744         0.7757   
          Std        0.0295           0.0319         0.0295   
Precision Mean       0.7496           0.7516         0.7496   
          Std        0.0417           0.0490         0.0417   
Recall    Mean       0.7757           0.7744         0.7757   
          Std        0.0295           0.0319         0.0295   
F1 score  Mean       0.7473           0.7512         0.7473   
          Std        0.0453           0.0442         0.0453   
ARI       Mean       0.1829           0.1925         0.1829   
          Std        0.0856           0.0836         0.0856   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7757   0.7757       0.7757   
          Std                           0.0295   0.0295       0.0295   
Precision Mean                          0.7496   0.7496       0.7496   
          Std                           0.0417   0.0417       0.0417   
Recall    Mean                          0.7757   0.7757       0.7757   
          Std                           0.0295   0.0295       0.0295   
F1 score  Mean                          0.7473   0.7473       0.7473   
          Std                           0.0453   0.0453       0.0453   
ARI       Mean                          0.1829   0.1829       0.1829   
          Std                           0.0856   0.0856       0.0856   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7757               0.7757   
          Std               0.0295               0.0295   
Precision Mean              0.7496               0.7496   
          Std               0.0417               0.0417   
Recall    Mean              0.7757               0.7757   
          Std               0.0295               0.0295   
F1 score  Mean              0.7473               0.7473   
          Std               0.0453               0.0453   
ARI       Mean              0.1829               0.1829   
          Std               0.0856               0.0856   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7757  
          Std                         0.0295  
Precision Mean                        0.7496  
          Std                         0.0417  
Recall    Mean                        0.7757  
          Std                         0.0295  
F1 score  Mean                        0.7473  
          Std                         0.0453  
ARI       Mean                        0.1829  
          Std                         0.0856

In [49]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7761           0.7723         0.7761   
          Std        0.0299           0.0306         0.0299   
Precision Mean       0.7635           0.7599         0.7635   
          Std        0.0353           0.0335         0.0353   
Recall    Mean       0.7761           0.7723         0.7761   
          Std        0.0299           0.0306         0.0299   
F1 score  Mean       0.7599           0.7583         0.7599   
          Std        0.0382           0.0354         0.0382   
ARI       Mean       0.2122           0.2079         0.2122   
          Std        0.0752           0.0682         0.0752   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7761   0.7761       0.7761   
          Std                           0.0299   0.0299       0.0299   
Precision Mean                          0.7635   0.7635       0.7635   
          Std                           0.0354   0.0353       0.0353   
Recall    Mean                          0.7761   0.7761       0.7761   
          Std                           0.0299   0.0299       0.0299   
F1 score  Mean                          0.7599   0.7599       0.7599   
          Std                           0.0382   0.0382       0.0382   
ARI       Mean                          0.2122   0.2122       0.2122   
          Std                           0.0751   0.0752       0.0752   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7761               0.7761   
          Std               0.0299               0.0299   
Precision Mean              0.7635               0.7635   
          Std               0.0353               0.0353   
Recall    Mean              0.7761               0.7761   
          Std               0.0299               0.0299   
F1 score  Mean              0.7599               0.7599   
          Std               0.0382               0.0382   
ARI       Mean              0.2122               0.2122   
          Std               0.0752               0.0752   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7761  
          Std                         0.0299  
Precision Mean                        0.7635  
          Std                         0.0353  
Recall    Mean                        0.7761  
          Std                         0.0299  
F1 score  Mean                        0.7599  
          Std                         0.0382  
ARI       Mean                        0.2122  
          Std                         0.0752

### Car-evaluation

In [53]:
import pandas as pd

with open('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/car+evaluation/car.names', 'r') as f:
    column_names = []
    for line in f:
        line = line.strip()
        if ':' in line and not line.startswith(('|', '#')):
            column_names.append(line.split(':')[0].strip())


df = pd.read_csv('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/car+evaluation/car.data', 
                 header=None, 
                 names=column_names,
                 delimiter=',')

df = df[['1. Title', '2. Sources', '(a) Creator', '(b) Donors', '(c) Date',
       '3. Past Usage','M. Bohanec and V. Rajkovic']]
df.columns = ['buying', 'maint', 'doors', 'persons','lug_boot','safety','class']

for column in df.columns:
    df[column], _ = pd.factorize(df[column])
    
X = df.drop('class', axis=1).to_numpy()
y = df['class'].to_numpy()

Columns: ['1. Title', '2. Sources', '(a) Creator', '(b) Donors', '(c) Date', '3. Past Usage', 'M. Bohanec and V. Rajkovic', 'B. Zupan, M. Bohanec, I. Bratko, J. Demsar', '4. Relevant Information Paragraph', '(M. Bohanec, V. Rajkovic', 'cars according to the following concept structure', 'concept (CAR), the model includes three intermediate concepts', 'these examples sets see http', 'attributes', '5. Number of Instances', '6. Number of Attributes', '7. Attribute Values', '8. Missing Attribute Values']


In [54]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7878           0.7887         0.7863   
          Std        0.0190           0.0190         0.0166   
Precision Mean       0.7300           0.7300         0.7330   
          Std        0.0271           0.0271         0.0278   
Recall    Mean       0.7878           0.7887         0.7863   
          Std        0.0190           0.0190         0.0166   
F1 score  Mean       0.7549           0.7554         0.7554   
          Std        0.0170           0.0172         0.0164   
ARI       Mean       0.5061           0.5088         0.4982   
          Std        0.0530           0.0527         0.0480   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7887   0.7896       0.7969   
          Std                           0.0190   0.0150       0.0147   
Precision Mean                          0.7300   0.7452       0.7670   
          Std                           0.0271   0.0306       0.0207   
Recall    Mean                          0.7887   0.7896       0.7969   
          Std                           0.0190   0.0150       0.0147   
F1 score  Mean                          0.7554   0.7627       0.7759   
          Std                           0.0172   0.0197       0.0164   
ARI       Mean                          0.5088   0.5008       0.5109   
          Std                           0.0527   0.0389       0.0379   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7887               0.7872   
          Std               0.0190               0.0181   
Precision Mean              0.7300               0.7301   
          Std               0.0271               0.0274   
Recall    Mean              0.7887               0.7872   
          Std               0.0190               0.0181   
F1 score  Mean              0.7554               0.7545   
          Std               0.0172               0.0161   
ARI       Mean              0.5088               0.5041   
          Std               0.0527               0.0532   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7887  
          Std                         0.0190  
Precision Mean                        0.7300  
          Std                         0.0271  
Recall    Mean                        0.7887  
          Std                         0.0190  
F1 score  Mean                        0.7554  
          Std                         0.0172  
ARI       Mean                        0.5088  
          Std                         0.0527

In [55]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8443           0.8339         0.8422   
          Std        0.0151           0.0183         0.0183   
Precision Mean       0.8451           0.8523         0.8434   
          Std        0.0223           0.0207         0.0214   
Recall    Mean       0.8443           0.8339         0.8422   
          Std        0.0151           0.0183         0.0183   
F1 score  Mean       0.8371           0.8308         0.8339   
          Std        0.0186           0.0198         0.0219   
ARI       Mean       0.6821           0.6506         0.6665   
          Std        0.0489           0.0554         0.0671   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8443   0.8363       0.8304   
          Std                           0.0151   0.0193       0.0182   
Precision Mean                          0.8458   0.8385       0.8319   
          Std                           0.0222   0.0181       0.0144   
Recall    Mean                          0.8443   0.8363       0.8304   
          Std                           0.0151   0.0193       0.0182   
F1 score  Mean                          0.8376   0.8254       0.8160   
          Std                           0.0181   0.0224       0.0176   
ARI       Mean                          0.6836   0.6351       0.6047   
          Std                           0.0464   0.0720       0.0553   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8443               0.8444   
          Std               0.0151               0.0152   
Precision Mean              0.8458               0.8445   
          Std               0.0222               0.0224   
Recall    Mean              0.8443               0.8444   
          Std               0.0151               0.0152   
F1 score  Mean              0.8376               0.8367   
          Std               0.0181               0.0188   
ARI       Mean              0.6836               0.6794   
          Std               0.0464               0.0534   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8443  
          Std                         0.0151  
Precision Mean                        0.8458  
          Std                         0.0222  
Recall    Mean                        0.8443  
          Std                         0.0151  
F1 score  Mean                        0.8376  
          Std                         0.0181  
ARI       Mean                        0.6836  
          Std                         0.0464

### Connectionist-bench-sonar

In [56]:
df = pd.read_csv('../Datasets/connectionist-bench-sonar.csv')
df['Label'] = df['Label'].map({'R':0, 'M':1})
X = df.drop('Label', axis=1).to_numpy()
y = df['Label'].to_numpy()

In [57]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7162           0.7211         0.7158   
          Std        0.0671           0.0674         0.0659   
Precision Mean       0.7292           0.7447         0.7301   
          Std        0.0703           0.0725         0.0678   
Recall    Mean       0.7162           0.7211         0.7158   
          Std        0.0671           0.0674         0.0659   
F1 score  Mean       0.7133           0.7166         0.7125   
          Std        0.0688           0.0690         0.0675   
ARI       Mean       0.1892           0.1978         0.1879   
          Std        0.1148           0.1196         0.1128   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7165   0.7135       0.7138   
          Std                           0.0667   0.0660       0.0661   
Precision Mean                          0.7300   0.7270       0.7275   
          Std                           0.0697   0.0681       0.0689   
Recall    Mean                          0.7165   0.7135       0.7138   
          Std                           0.0667   0.0660       0.0661   
F1 score  Mean                          0.7136   0.7106       0.7108   
          Std                           0.0685   0.0674       0.0679   
ARI       Mean                          0.1897   0.1839       0.1846   
          Std                           0.1141   0.1122       0.1125   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7138               0.7158   
          Std               0.0661               0.0681   
Precision Mean              0.7274               0.7294   
          Std               0.0690               0.0707   
Recall    Mean              0.7138               0.7158   
          Std               0.0661               0.0681   
F1 score  Mean              0.7108               0.7128   
          Std               0.0679               0.0698   
ARI       Mean              0.1846               0.1891   
          Std               0.1125               0.1172   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7127  
          Std                         0.0651  
Precision Mean                        0.7263  
          Std                         0.0679  
Recall    Mean                        0.7127  
          Std                         0.0651  
F1 score  Mean                        0.7097  
          Std                         0.0668  
ARI       Mean                        0.1821  
          Std                         0.1101

In [58]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7131           0.7235         0.7135   
          Std        0.0655           0.0696         0.0662   
Precision Mean       0.7229           0.7366         0.7239   
          Std        0.0655           0.0685         0.0650   
Recall    Mean       0.7131           0.7235         0.7135   
          Std        0.0655           0.0696         0.0662   
F1 score  Mean       0.7118           0.7220         0.7119   
          Std        0.0667           0.0697         0.0676   
ARI       Mean       0.1830           0.2037         0.1840   
          Std        0.1125           0.1232         0.1173   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7115   0.7119       0.7108   
          Std                           0.0658   0.0635       0.0659   
Precision Mean                          0.7215   0.7216       0.7206   
          Std                           0.0663   0.0630       0.0657   
Recall    Mean                          0.7115   0.7119       0.7108   
          Std                           0.0658   0.0635       0.0659   
F1 score  Mean                          0.7103   0.7106       0.7095   
          Std                           0.0669   0.0648       0.0672   
ARI       Mean                          0.1804   0.1799       0.1792   
          Std                           0.1142   0.1106       0.1126   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7131               0.7127   
          Std               0.0647               0.0644   
Precision Mean              0.7222               0.7228   
          Std               0.0646               0.0641   
Recall    Mean              0.7131               0.7127   
          Std               0.0647               0.0644   
F1 score  Mean              0.7118               0.7114   
          Std               0.0659               0.0657   
ARI       Mean              0.1825               0.1817   
          Std               0.1112               0.1127   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7119  
          Std                         0.0638  
Precision Mean                        0.7214  
          Std                         0.0632  
Recall    Mean                        0.7119  
          Std                         0.0638  
F1 score  Mean                        0.7106  
          Std                         0.0652  
ARI       Mean                        0.1801  
          Std                         0.1110

### Contraceptive-method-choice

In [60]:
import pandas as pd

with open('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/contraceptive+method+choice/cmc.names', 'r') as f:
    column_names = []
    for line in f:
        line = line.strip()
        if ':' in line and not line.startswith(('|', '#')):
            column_names.append(line.split(':')[0].strip())

print("Columns:", column_names)

df = pd.read_csv('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/contraceptive+method+choice/cmc.data', 
                 header=None, 
                 names=column_names,
                 delimiter=',')

df = df[['1. Title', '2. Sources', '(a) Origin', '(b) Creator', '(c) Donor', '(c) Date', '3. Past Usage', '(ftp', '(http', '4. Relevant Information']]
df.columns = ['w_age', 'w_education','h_education', 'N_children','w_religion','w_working','h_occupation','sol_index','media_exposure','class']

X = df.drop('class', axis=1).to_numpy()
y = df['class'].to_numpy()

Columns: ['1. Title', '2. Sources', '(a) Origin', '(b) Creator', '(c) Donor', '(c) Date', '3. Past Usage', '(ftp', '(http', '4. Relevant Information', '5. Number of Instances', '6. Number of Attributes', '7. Attribute Information', '8. Missing Attribute Values']


In [61]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.5118           0.5016         0.5128   
          Std        0.0269           0.0293         0.0291   
Precision Mean       0.5677           0.5461         0.5374   
          Std        0.0564           0.0773         0.0677   
Recall    Mean       0.5118           0.5016         0.5128   
          Std        0.0269           0.0293         0.0291   
F1 score  Mean       0.4992           0.4855         0.4894   
          Std        0.0317           0.0428         0.0389   
ARI       Mean       0.0676           0.0605         0.0687   
          Std        0.0229           0.0247         0.0284   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.5131   0.5133       0.5130   
          Std                           0.0279   0.0316       0.0296   
Precision Mean                          0.5754   0.5147       0.5043   
          Std                           0.0524   0.0656       0.0734   
Recall    Mean                          0.5131   0.5133       0.5130   
          Std                           0.0279   0.0316       0.0296   
F1 score  Mean                          0.5052   0.4886       0.4873   
          Std                           0.0348   0.0428       0.0437   
ARI       Mean                          0.0698   0.0735       0.0732   
          Std                           0.0244   0.0279       0.0258   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5215               0.5109   
          Std               0.0246               0.0257   
Precision Mean              0.5677               0.5604   
          Std               0.0604               0.0581   
Recall    Mean              0.5215               0.5109   
          Std               0.0246               0.0257   
F1 score  Mean              0.5146               0.4942   
          Std               0.0387               0.0343   
ARI       Mean              0.0808               0.0659   
          Std               0.0223               0.0230   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5143  
          Std                         0.0267  
Precision Mean                        0.5764  
          Std                         0.0510  
Recall    Mean                        0.5143  
          Std                         0.0267  
F1 score  Mean                        0.5069  
          Std                         0.0345  
ARI       Mean                        0.0709  
          Std                         0.0241

In [62]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.5512           0.5431         0.5433   
          Std        0.0237           0.0258         0.0252   
Precision Mean       0.5648           0.5619         0.5594   
          Std        0.0284           0.0367         0.0291   
Recall    Mean       0.5512           0.5431         0.5433   
          Std        0.0237           0.0258         0.0252   
F1 score  Mean       0.5455           0.5370         0.5363   
          Std        0.0244           0.0322         0.0258   
ARI       Mean       0.1115           0.1020         0.1035   
          Std        0.0266           0.0298         0.0294   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.5497   0.5373       0.5306   
          Std                           0.0227   0.0241       0.0268   
Precision Mean                          0.5618   0.5600       0.5565   
          Std                           0.0260   0.0295       0.0401   
Recall    Mean                          0.5497   0.5373       0.5306   
          Std                           0.0227   0.0241       0.0268   
F1 score  Mean                          0.5450   0.5306       0.5220   
          Std                           0.0242   0.0238       0.0305   
ARI       Mean                          0.1111   0.0953       0.0886   
          Std                           0.0260   0.0262       0.0266   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5492               0.5477   
          Std               0.0223               0.0246   
Precision Mean              0.5652               0.5625   
          Std               0.0263               0.0291   
Recall    Mean              0.5492               0.5477   
          Std               0.0223               0.0246   
F1 score  Mean              0.5469               0.5419   
          Std               0.0232               0.0250   
ARI       Mean              0.1115               0.1084   
          Std               0.0259               0.0281   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5524  
          Std                         0.0209  
Precision Mean                        0.5628  
          Std                         0.0262  
Recall    Mean                        0.5524  
          Std                         0.0209  
F1 score  Mean                        0.5472  
          Std                         0.0232  
ARI       Mean                        0.1141  
          Std                         0.0244

### Haberman-survival

In [63]:
import pandas as pd

with open('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/haberman+s+survival/haberman.names', 'r') as f:
    column_names = []
    for line in f:
        line = line.strip()
        if ':' in line and not line.startswith(('|', '#')):
            column_names.append(line.split(':')[0].strip())

df = pd.read_csv('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/haberman+s+survival/haberman.data', 
                 header=None, 
                 names=column_names,
                 delimiter=',')

df = df[['1. Title', '2. Sources', '(a) Donor', '(b) Date']]
df.columns = ['age', 'operation_year','positive_auxillary_nodes','survival_status']

X = df.drop('survival_status', axis=1).to_numpy()
y = df['survival_status'].to_numpy()

In [64]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7343           0.7343         0.7343   
          Std        0.0462           0.0465         0.0462   
Precision Mean       0.7150           0.7172         0.7150   
          Std        0.0697           0.0695         0.0697   
Recall    Mean       0.7343           0.7343         0.7343   
          Std        0.0462           0.0465         0.0462   
F1 score  Mean       0.7095           0.7107         0.7095   
          Std        0.0600           0.0631         0.0600   
ARI       Mean       0.1244           0.1289         0.1244   
          Std        0.0856           0.0914         0.0856   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7343   0.7348       0.7343   
          Std                           0.0462   0.0459       0.0462   
Precision Mean                          0.7150   0.7155       0.7150   
          Std                           0.0697   0.0698       0.0697   
Recall    Mean                          0.7343   0.7348       0.7343   
          Std                           0.0462   0.0459       0.0462   
F1 score  Mean                          0.7095   0.7100       0.7095   
          Std                           0.0600   0.0601       0.0600   
ARI       Mean                          0.1244   0.1253       0.1244   
          Std                           0.0856   0.0857       0.0856   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7343               0.7343   
          Std               0.0462               0.0462   
Precision Mean              0.7150               0.7150   
          Std               0.0697               0.0697   
Recall    Mean              0.7343               0.7343   
          Std               0.0462               0.0462   
F1 score  Mean              0.7095               0.7095   
          Std               0.0600               0.0600   
ARI       Mean              0.1244               0.1244   
          Std               0.0856               0.0856   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7343  
          Std                         0.0462  
Precision Mean                        0.7150  
          Std                         0.0697  
Recall    Mean                        0.7343  
          Std                         0.0462  
F1 score  Mean                        0.7095  
          Std                         0.0600  
ARI       Mean                        0.1244  
          Std                         0.0856

In [65]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7322           0.7247         0.7319   
          Std        0.0472           0.0439         0.0470   
Precision Mean       0.7239           0.7050         0.7237   
          Std        0.0556           0.0587         0.0555   
Recall    Mean       0.7322           0.7247         0.7319   
          Std        0.0472           0.0439         0.0470   
F1 score  Mean       0.7178           0.7067         0.7176   
          Std        0.0493           0.0518         0.0491   
ARI       Mean       0.1375           0.1141         0.1370   
          Std        0.0776           0.0830         0.0772   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7306   0.7319       0.7319   
          Std                           0.0467   0.0470       0.0470   
Precision Mean                          0.7216   0.7237       0.7237   
          Std                           0.0561   0.0555       0.0555   
Recall    Mean                          0.7306   0.7319       0.7319   
          Std                           0.0467   0.0470       0.0470   
F1 score  Mean                          0.7156   0.7176       0.7176   
          Std                           0.0497   0.0491       0.0491   
ARI       Mean                          0.1334   0.1370       0.1370   
          Std                           0.0766   0.0772       0.0772   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7317               0.7317   
          Std               0.0472               0.0472   
Precision Mean              0.7233               0.7233   
          Std               0.0555               0.0555   
Recall    Mean              0.7317               0.7317   
          Std               0.0472               0.0472   
F1 score  Mean              0.7173               0.7173   
          Std               0.0491               0.0491   
ARI       Mean              0.1363               0.1363   
          Std               0.0780               0.0780   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7317  
          Std                         0.0472  
Precision Mean                        0.7233  
          Std                         0.0555  
Recall    Mean                        0.7317  
          Std                         0.0472  
F1 score  Mean                        0.7173  
          Std                         0.0491  
ARI       Mean                        0.1363  
          Std                         0.0780

### Hayes-roth

In [66]:
df = pd.read_csv('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/hayes+roth/hayes-roth.data', header=None)
df.columns = ['name','hobby','age','educational level','marital status','class']

X = df.drop(['class','name'],axis=1).to_numpy()
y = df['class'].to_numpy()

In [67]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.5503           0.5528         0.5370   
          Std        0.0628           0.0602         0.0437   
Precision Mean       0.3730           0.3704         0.5486   
          Std        0.0605           0.0585         0.1309   
Recall    Mean       0.5503           0.5528         0.5370   
          Std        0.0628           0.0602         0.0437   
F1 score  Mean       0.4243           0.4241         0.5131   
          Std        0.0630           0.0630         0.0843   
ARI       Mean       0.4584           0.4657         0.1953   
          Std        0.0893           0.0698         0.1582   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.5528   0.5364       0.5479   
          Std                           0.0602   0.0536       0.0635   
Precision Mean                          0.3704   0.5613       0.5848   
          Std                           0.0585   0.1251       0.1242   
Recall    Mean                          0.5528   0.5364       0.5479   
          Std                           0.0602   0.0536       0.0635   
F1 score  Mean                          0.4241   0.5189       0.5337   
          Std                           0.0630   0.0886       0.0895   
ARI       Mean                          0.4657   0.1766       0.1751   
          Std                           0.0698   0.1362       0.1309   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5528               0.5358   
          Std               0.0602               0.0443   
Precision Mean              0.3704               0.5169   
          Std               0.0585               0.1364   
Recall    Mean              0.5528               0.5358   
          Std               0.0602               0.0443   
F1 score  Mean              0.4241               0.4952   
          Std               0.0630               0.0843   
ARI       Mean              0.4657               0.2334   
          Std               0.0698               0.1777   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5528  
          Std                         0.0602  
Precision Mean                        0.3704  
          Std                         0.0585  
Recall    Mean                        0.5528  
          Std                         0.0602  
F1 score  Mean                        0.4241  
          Std                         0.0630  
ARI       Mean                        0.4657  
          Std                         0.0698

In [68]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6564           0.6564         0.5952   
          Std        0.0448           0.0448         0.0907   
Precision Mean       0.6759           0.6759         0.6597   
          Std        0.0625           0.0625         0.0830   
Recall    Mean       0.6564           0.6564         0.5952   
          Std        0.0448           0.0448         0.0907   
F1 score  Mean       0.6551           0.6551         0.5896   
          Std        0.0476           0.0476         0.0942   
ARI       Mean       0.3449           0.3449         0.2539   
          Std        0.0757           0.0757         0.1180   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6564   0.6194       0.6303   
          Std                           0.0448   0.0964       0.0913   
Precision Mean                          0.6759   0.7019       0.7028   
          Std                           0.0625   0.0920       0.0964   
Recall    Mean                          0.6564   0.6194       0.6303   
          Std                           0.0448   0.0964       0.0913   
F1 score  Mean                          0.6551   0.6095       0.6200   
          Std                           0.0476   0.0967       0.0941   
ARI       Mean                          0.3449   0.2550       0.2709   
          Std                           0.0757   0.1244       0.1087   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6564               0.6000   
          Std               0.0448               0.0831   
Precision Mean              0.6759               0.6529   
          Std               0.0625               0.0763   
Recall    Mean              0.6564               0.6000   
          Std               0.0448               0.0831   
F1 score  Mean              0.6551               0.5979   
          Std               0.0476               0.0872   
ARI       Mean              0.3449               0.2583   
          Std               0.0757               0.1053   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6564  
          Std                         0.0448  
Precision Mean                        0.6759  
          Std                         0.0625  
Recall    Mean                        0.6564  
          Std                         0.0448  
F1 score  Mean                        0.6551  
          Std                         0.0476  
ARI       Mean                        0.3449  
          Std                         0.0757

### Heart-disease-Cleveland

In [69]:
df = pd.read_csv('../Datasets/Heart_disease_cleveland_new.csv')
X = df.drop('target', axis=1).to_numpy()
y = df['target'].to_numpy()

In [70]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7858           0.7821         0.7858   
          Std        0.0412           0.0454         0.0412   
Precision Mean       0.7904           0.7885         0.7904   
          Std        0.0429           0.0436         0.0429   
Recall    Mean       0.7858           0.7821         0.7858   
          Std        0.0412           0.0454         0.0412   
F1 score  Mean       0.7846           0.7804         0.7846   
          Std        0.0412           0.0463         0.0412   
ARI       Mean       0.3244           0.3175         0.3244   
          Std        0.0955           0.1003         0.0955   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7858   0.7858       0.7858   
          Std                           0.0412   0.0412       0.0412   
Precision Mean                          0.7904   0.7904       0.7904   
          Std                           0.0429   0.0429       0.0429   
Recall    Mean                          0.7858   0.7858       0.7858   
          Std                           0.0412   0.0412       0.0412   
F1 score  Mean                          0.7846   0.7846       0.7846   
          Std                           0.0412   0.0412       0.0412   
ARI       Mean                          0.3244   0.3244       0.3244   
          Std                           0.0955   0.0955       0.0955   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7858               0.7858   
          Std               0.0412               0.0412   
Precision Mean              0.7904               0.7904   
          Std               0.0429               0.0429   
Recall    Mean              0.7858               0.7858   
          Std               0.0412               0.0412   
F1 score  Mean              0.7846               0.7846   
          Std               0.0412               0.0412   
ARI       Mean              0.3244               0.3244   
          Std               0.0955               0.0955   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7858  
          Std                         0.0412  
Precision Mean                        0.7905  
          Std                         0.0429  
Recall    Mean                        0.7858  
          Std                         0.0412  
F1 score  Mean                        0.7846  
          Std                         0.0412  
ARI       Mean                        0.3244  
          Std                         0.0955

In [71]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.7466           0.7605         0.7479   
          Std        0.0427           0.0439         0.0441   
Precision Mean       0.7565           0.7708         0.7577   
          Std        0.0433           0.0441         0.0445   
Recall    Mean       0.7466           0.7605         0.7479   
          Std        0.0427           0.0439         0.0441   
F1 score  Mean       0.7444           0.7578         0.7457   
          Std        0.0433           0.0444         0.0448   
ARI       Mean       0.2404           0.2695         0.2436   
          Std        0.0823           0.0933         0.0862   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.7482   0.7482       0.7482   
          Std                           0.0447   0.0442       0.0442   
Precision Mean                          0.7581   0.7580       0.7580   
          Std                           0.0451   0.0445       0.0445   
Recall    Mean                          0.7482   0.7482       0.7482   
          Std                           0.0447   0.0442       0.0442   
F1 score  Mean                          0.7460   0.7460       0.7460   
          Std                           0.0453   0.0449       0.0449   
ARI       Mean                          0.2443   0.2441       0.2441   
          Std                           0.0868   0.0863       0.0863   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.7476               0.7476   
          Std               0.0438               0.0438   
Precision Mean              0.7574               0.7574   
          Std               0.0443               0.0443   
Recall    Mean              0.7476               0.7476   
          Std               0.0438               0.0438   
F1 score  Mean              0.7454               0.7454   
          Std               0.0445               0.0445   
ARI       Mean              0.2430               0.2430   
          Std               0.0855               0.0855   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.7479  
          Std                         0.0443  
Precision Mean                        0.7577  
          Std                         0.0447  
Recall    Mean                        0.7479  
          Std                         0.0443  
F1 score  Mean                        0.7457  
          Std                         0.0449  
ARI       Mean                        0.2436  
          Std                         0.0865

### Hepatitis

In [72]:
df = pd.read_csv('../Datasets/hepatitis_csv.csv')
df = df.dropna()
df = df.replace({True: 1, False: 0})
df['sex'] = df['sex'].map({'female': 0, 'male': 1})
df['class'] = df['class'].map({'live': 1, 'die': 0})

X = df.drop('class',axis=1).to_numpy()
y = df['class'].to_numpy()

/var/folders/2q/k8j7_x7j65q07cgb4j_r745h0000gn/T/ipykernel_55592/1941710694.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [73]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8110           0.8150         0.8100   
          Std        0.0666           0.0743         0.0700   
Precision Mean       0.8166           0.8427         0.8134   
          Std        0.0977           0.0893         0.1014   
Recall    Mean       0.8110           0.8150         0.8100   
          Std        0.0666           0.0743         0.0700   
F1 score  Mean       0.8071           0.8155         0.8050   
          Std        0.0783           0.0809         0.0822   
ARI       Mean       0.1992           0.2458         0.1947   
          Std        0.1894           0.2302         0.1916   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8080   0.8090       0.8100   
          Std                           0.0681   0.0676       0.0693   
Precision Mean                          0.8137   0.8152       0.8138   
          Std                           0.1001   0.0991       0.1002   
Recall    Mean                          0.8080   0.8090       0.8100   
          Std                           0.0681   0.0676       0.0693   
F1 score  Mean                          0.8038   0.8053       0.8049   
          Std                           0.0805   0.0794       0.0814   
ARI       Mean                          0.1937   0.1970       0.1941   
          Std                           0.1905   0.1902       0.1901   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8130               0.8170   
          Std               0.0699               0.0704   
Precision Mean              0.8215               0.8245   
          Std               0.0988               0.0973   
Recall    Mean              0.8130               0.8170   
          Std               0.0699               0.0704   
F1 score  Mean              0.8098               0.8136   
          Std               0.0808               0.0798   
ARI       Mean              0.2141               0.2256   
          Std               0.1930               0.1989   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8090  
          Std                         0.0683  
Precision Mean                        0.8138  
          Std                         0.1002  
Recall    Mean                        0.8090  
          Std                         0.0683  
F1 score  Mean                        0.8044  
          Std                         0.0809  
ARI       Mean                        0.1938  
          Std                         0.1905

In [74]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8100           0.8120         0.8080   
          Std        0.0686           0.0637         0.0717   
Precision Mean       0.8280           0.8377         0.8242   
          Std        0.0919           0.0864         0.0964   
Recall    Mean       0.8100           0.8120         0.8080   
          Std        0.0686           0.0637         0.0717   
F1 score  Mean       0.8129           0.8131         0.8101   
          Std        0.0763           0.0746         0.0807   
ARI       Mean       0.2294           0.2218         0.2242   
          Std        0.1911           0.2077         0.1944   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8060   0.8080       0.8090   
          Std                           0.0712   0.0703       0.0705   
Precision Mean                          0.8233   0.8267       0.8252   
          Std                           0.0961   0.0935       0.0949   
Recall    Mean                          0.8060   0.8080       0.8090   
          Std                           0.0712   0.0703       0.0705   
F1 score  Mean                          0.8080   0.8109       0.8108   
          Std                           0.0797   0.0777       0.0797   
ARI       Mean                          0.2178   0.2260       0.2257   
          Std                           0.1987   0.1926       0.1931   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8060               0.8090   
          Std               0.0712               0.0705   
Precision Mean              0.8257               0.8283   
          Std               0.0961               0.0941   
Recall    Mean              0.8060               0.8090   
          Std               0.0712               0.0705   
F1 score  Mean              0.8088               0.8118   
          Std               0.0794               0.0771   
ARI       Mean              0.2221               0.2296   
          Std               0.1943               0.1920   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8080  
          Std                         0.0703  
Precision Mean                        0.8252  
          Std                         0.0948  
Recall    Mean                        0.8080  
          Std                         0.0703  
F1 score  Mean                        0.8101  
          Std                         0.0791  
ARI       Mean                        0.2240  
          Std                         0.1931

### Ionosphere

In [75]:
df = pd.read_csv('/Users/user/HSE 24:25/A Family of Classifiying criteria/Datasets/ionosphere/ionosphere.data', 
                 header=None,
                 delimiter=',')
df = pd.get_dummies(df, drop_first=True, dtype=int)
X = df.drop('34_g', axis=1).to_numpy()
y = df['34_g'].to_numpy()

In [76]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8866           0.8973         0.8870   
          Std        0.0418           0.0382         0.0408   
Precision Mean       0.8924           0.9018         0.8931   
          Std        0.0368           0.0332         0.0351   
Recall    Mean       0.8866           0.8973         0.8870   
          Std        0.0418           0.0382         0.0408   
F1 score  Mean       0.8857           0.8967         0.8861   
          Std        0.0445           0.0405         0.0435   
ARI       Mean       0.5963           0.6293         0.5974   
          Std        0.1265           0.1167         0.1242   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8868   0.8866       0.8866   
          Std                           0.0415   0.0418       0.0418   
Precision Mean                          0.8927   0.8924       0.8924   
          Std                           0.0362   0.0368       0.0368   
Recall    Mean                          0.8868   0.8866       0.8866   
          Std                           0.0415   0.0418       0.0418   
F1 score  Mean                          0.8859   0.8857       0.8857   
          Std                           0.0442   0.0445       0.0445   
ARI       Mean                          0.5970   0.5963       0.5963   
          Std                           0.1255   0.1265       0.1265   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8859               0.8866   
          Std               0.0421               0.0418   
Precision Mean              0.8917               0.8924   
          Std               0.0373               0.0368   
Recall    Mean              0.8859               0.8866   
          Std               0.0421               0.0418   
F1 score  Mean              0.8850               0.8857   
          Std               0.0448               0.0445   
ARI       Mean              0.5942               0.5963   
          Std               0.1271               0.1265   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8866  
          Std                         0.0418  
Precision Mean                        0.8924  
          Std                         0.0368  
Recall    Mean                        0.8866  
          Std                         0.0418  
F1 score  Mean                        0.8857  
          Std                         0.0445  
ARI       Mean                        0.5963  
          Std                         0.1265

In [77]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8820           0.8789         0.8816   
          Std        0.0322           0.0387         0.0315   
Precision Mean       0.8860           0.8846         0.8857   
          Std        0.0323           0.0380         0.0313   
Recall    Mean       0.8820           0.8789         0.8816   
          Std        0.0322           0.0387         0.0315   
F1 score  Mean       0.8795           0.8758         0.8791   
          Std        0.0335           0.0396         0.0327   
ARI       Mean       0.5757           0.5676         0.5742   
          Std        0.1016           0.1178         0.1002   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8820   0.8816       0.8816   
          Std                           0.0318   0.0322       0.0322   
Precision Mean                          0.8862   0.8854       0.8854   
          Std                           0.0316   0.0323       0.0323   
Recall    Mean                          0.8820   0.8816       0.8816   
          Std                           0.0318   0.0322       0.0322   
F1 score  Mean                          0.8795   0.8791       0.8791   
          Std                           0.0330   0.0334       0.0334   
ARI       Mean                          0.5756   0.5743       0.5743   
          Std                           0.0998   0.1016       0.1016   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8811               0.8820   
          Std               0.0321               0.0321   
Precision Mean              0.8851               0.8858   
          Std               0.0321               0.0324   
Recall    Mean              0.8811               0.8820   
          Std               0.0321               0.0321   
F1 score  Mean              0.8785               0.8796   
          Std               0.0333               0.0333   
ARI       Mean              0.5728               0.5757   
          Std               0.1013               0.1013   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8823  
          Std                         0.0317  
Precision Mean                        0.8861  
          Std                         0.0319  
Recall    Mean                        0.8823  
          Std                         0.0317  
F1 score  Mean                        0.8798  
          Std                         0.0330  
ARI       Mean                        0.5763  
          Std                         0.1003

### Mammographic Mass

In [78]:
df = pd.read_csv('../Datasets/Mammographic Mass.csv')
X = df.drop('Severity', axis=1).to_numpy()
y = df['Severity'].to_numpy()

In [79]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8430           0.8393         0.8428   
          Std        0.0228           0.0213         0.0226   
Precision Mean       0.8485           0.8448         0.8484   
          Std        0.0214           0.0199         0.0212   
Recall    Mean       0.8430           0.8393         0.8428   
          Std        0.0228           0.0213         0.0226   
F1 score  Mean       0.8421           0.8384         0.8419   
          Std        0.0231           0.0216         0.0229   
ARI       Mean       0.4700           0.4597         0.4695   
          Std        0.0618           0.0569         0.0613   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8428   0.8428       0.8428   
          Std                           0.0226   0.0226       0.0226   
Precision Mean                          0.8484   0.8484       0.8484   
          Std                           0.0212   0.0212       0.0212   
Recall    Mean                          0.8428   0.8428       0.8428   
          Std                           0.0226   0.0226       0.0226   
F1 score  Mean                          0.8419   0.8419       0.8419   
          Std                           0.0229   0.0229       0.0229   
ARI       Mean                          0.4695   0.4695       0.4695   
          Std                           0.0613   0.0613       0.0613   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8428               0.8428   
          Std               0.0226               0.0226   
Precision Mean              0.8484               0.8484   
          Std               0.0212               0.0212   
Recall    Mean              0.8428               0.8428   
          Std               0.0226               0.0226   
F1 score  Mean              0.8419               0.8419   
          Std               0.0229               0.0229   
ARI       Mean              0.4695               0.4695   
          Std               0.0613               0.0613   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8428  
          Std                         0.0226  
Precision Mean                        0.8484  
          Std                         0.0212  
Recall    Mean                        0.8428  
          Std                         0.0226  
F1 score  Mean                        0.8419  
          Std                         0.0229  
ARI       Mean                        0.4695  
          Std                         0.0613

In [80]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8262           0.8313         0.8261   
          Std        0.0232           0.0243         0.0233   
Precision Mean       0.8314           0.8362         0.8312   
          Std        0.0228           0.0236         0.0231   
Recall    Mean       0.8262           0.8313         0.8261   
          Std        0.0232           0.0243         0.0233   
F1 score  Mean       0.8257           0.8307         0.8255   
          Std        0.0232           0.0244         0.0234   
ARI       Mean       0.4251           0.4388         0.4247   
          Std        0.0596           0.0631         0.0601   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8262   0.8262       0.8262   
          Std                           0.0231   0.0232       0.0232   
Precision Mean                          0.8314   0.8313       0.8313   
          Std                           0.0228   0.0228       0.0228   
Recall    Mean                          0.8262   0.8262       0.8262   
          Std                           0.0231   0.0232       0.0232   
F1 score  Mean                          0.8256   0.8256       0.8256   
          Std                           0.0231   0.0232       0.0232   
ARI       Mean                          0.4249   0.4249       0.4249   
          Std                           0.0594   0.0596       0.0596   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8262               0.8262   
          Std               0.0232               0.0233   
Precision Mean              0.8313               0.8314   
          Std               0.0228               0.0229   
Recall    Mean              0.8262               0.8262   
          Std               0.0232               0.0233   
F1 score  Mean              0.8256               0.8257   
          Std               0.0232               0.0233   
ARI       Mean              0.4249               0.4252   
          Std               0.0596               0.0599   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8262  
          Std                         0.0232  
Precision Mean                        0.8313  
          Std                         0.0228  
Recall    Mean                        0.8262  
          Std                         0.0232  
F1 score  Mean                        0.8256  
          Std                         0.0232  
ARI       Mean                        0.4249  
          Std                         0.0596

### Monks

In [81]:
df = pd.read_csv('../Datasets/monk.csv')
df = df.drop('id', axis=1)
X = df.drop("'class'", axis=1).to_numpy()
y = df["'class'"].to_numpy()

In [82]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6119           0.6156         0.6119   
          Std        0.0352           0.0359         0.0352   
Precision Mean       0.5455           0.5433         0.5455   
          Std        0.0815           0.0840         0.0815   
Recall    Mean       0.6119           0.6156         0.6119   
          Std        0.0352           0.0359         0.0352   
F1 score  Mean       0.5585           0.5568         0.5585   
          Std        0.0511           0.0515         0.0511   
ARI       Mean       0.0114           0.0117         0.0114   
          Std        0.0227           0.0225         0.0227   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6119   0.6119       0.6119   
          Std                           0.0352   0.0352       0.0352   
Precision Mean                          0.5455   0.5455       0.5455   
          Std                           0.0815   0.0815       0.0815   
Recall    Mean                          0.6119   0.6119       0.6119   
          Std                           0.0352   0.0352       0.0352   
F1 score  Mean                          0.5585   0.5585       0.5585   
          Std                           0.0511   0.0511       0.0511   
ARI       Mean                          0.0114   0.0114       0.0114   
          Std                           0.0227   0.0227       0.0227   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6119               0.6119   
          Std               0.0352               0.0352   
Precision Mean              0.5455               0.5455   
          Std               0.0815               0.0815   
Recall    Mean              0.6119               0.6119   
          Std               0.0352               0.0352   
F1 score  Mean              0.5585               0.5585   
          Std               0.0511               0.0511   
ARI       Mean              0.0114               0.0114   
          Std               0.0227               0.0227   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6119  
          Std                         0.0352  
Precision Mean                        0.5455  
          Std                         0.0815  
Recall    Mean                        0.6119  
          Std                         0.0352  
F1 score  Mean                        0.5585  
          Std                         0.0511  
ARI       Mean                        0.0114  
          Std                         0.0227

In [83]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6151           0.6126         0.6151   
          Std        0.0376           0.0351         0.0376   
Precision Mean       0.6137           0.6047         0.6137   
          Std        0.0468           0.0485         0.0468   
Recall    Mean       0.6151           0.6126         0.6151   
          Std        0.0376           0.0351         0.0376   
F1 score  Mean       0.6022           0.5946         0.6022   
          Std        0.0449           0.0450         0.0449   
ARI       Mean       0.0387           0.0321         0.0387   
          Std        0.0312           0.0309         0.0312   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6151   0.6151       0.6151   
          Std                           0.0376   0.0376       0.0376   
Precision Mean                          0.6137   0.6137       0.6137   
          Std                           0.0468   0.0468       0.0468   
Recall    Mean                          0.6151   0.6151       0.6151   
          Std                           0.0376   0.0376       0.0376   
F1 score  Mean                          0.6022   0.6022       0.6022   
          Std                           0.0449   0.0449       0.0449   
ARI       Mean                          0.0387   0.0387       0.0387   
          Std                           0.0312   0.0312       0.0312   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6151               0.6151   
          Std               0.0376               0.0376   
Precision Mean              0.6137               0.6137   
          Std               0.0468               0.0468   
Recall    Mean              0.6151               0.6151   
          Std               0.0376               0.0376   
F1 score  Mean              0.6022               0.6022   
          Std               0.0449               0.0449   
ARI       Mean              0.0387               0.0387   
          Std               0.0312               0.0312   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6151  
          Std                         0.0376  
Precision Mean                        0.6137  
          Std                         0.0468  
Recall    Mean                        0.6151  
          Std                         0.0376  
F1 score  Mean                        0.6022  
          Std                         0.0449  
ARI       Mean                        0.0387  
          Std                         0.0312

### Spambase

In [6]:
df = pd.read_csv('../Datasets/spambase.csv')
X = df.drop('spam', axis=1).to_numpy()
y = df['spam'].to_numpy()

In [7]:
compare_metrics_train_test(max_depth=3, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 3)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8820           0.8693         0.8820   
          Std        0.0111           0.0085         0.0111   
Precision Mean       0.8832           0.8757         0.8832   
          Std        0.0105           0.0084         0.0105   
Recall    Mean       0.8820           0.8693         0.8820   
          Std        0.0111           0.0085         0.0111   
F1 score  Mean       0.8810           0.8667         0.8810   
          Std        0.0113           0.0094         0.0113   
ARI       Mean       0.5824           0.5430         0.5824   
          Std        0.0337           0.0258         0.0337   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8820   0.8820       0.8820   
          Std                           0.0111   0.0111       0.0111   
Precision Mean                          0.8832   0.8832       0.8832   
          Std                           0.0105   0.0105       0.0105   
Recall    Mean                          0.8820   0.8820       0.8820   
          Std                           0.0111   0.0111       0.0111   
F1 score  Mean                          0.8810   0.8810       0.8810   
          Std                           0.0113   0.0113       0.0113   
ARI       Mean                          0.5824   0.5824       0.5824   
          Std                           0.0337   0.0337       0.0337   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8820               0.8820   
          Std               0.0111               0.0111   
Precision Mean              0.8832               0.8832   
          Std               0.0105               0.0105   
Recall    Mean              0.8820               0.8820   
          Std               0.0111               0.0111   
F1 score  Mean              0.8810               0.8810   
          Std               0.0113               0.0113   
ARI       Mean              0.5824               0.5824   
          Std               0.0337               0.0337   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8820  
          Std                         0.0111  
Precision Mean                        0.8832  
          Std                         0.0105  
Recall    Mean                        0.8820  
          Std                         0.0111  
F1 score  Mean                        0.8810  
          Std                         0.0113  
ARI       Mean                        0.5824  
          Std                         0.0337

In [8]:
compare_metrics_train_test(max_depth=4, X=X, y=y)


N, V, k, alpha, nmin, max_depth = (None, None, None, None, None, 4)


b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.8976           0.8982         0.8976   
          Std        0.0104           0.0086         0.0104   
Precision Mean       0.8995           0.8997         0.8995   
          Std        0.0105           0.0084         0.0105   
Recall    Mean       0.8976           0.8982         0.8976   
          Std        0.0104           0.0086         0.0104   
F1 score  Mean       0.8964           0.8969         0.8964   
          Std        0.0104           0.0087         0.0104   
ARI       Mean       0.6310           0.6326         0.6310   
          Std        0.0330           0.0272         0.0330   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.8976   0.8976       0.8976   
          Std                           0.0104   0.0104       0.0104   
Precision Mean                          0.8995   0.8995       0.8995   
          Std                           0.0105   0.0105       0.0105   
Recall    Mean                          0.8976   0.8976       0.8976   
          Std                           0.0104   0.0104       0.0104   
F1 score  Mean                          0.8964   0.8964       0.8964   
          Std                           0.0104   0.0104       0.0104   
ARI       Mean                          0.6309   0.6310       0.6310   
          Std                           0.0330   0.0330       0.0330   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.8976               0.8976   
          Std               0.0104               0.0104   
Precision Mean              0.8995               0.8996   
          Std               0.0105               0.0105   
Recall    Mean              0.8976               0.8976   
          Std               0.0104               0.0104   
F1 score  Mean              0.8964               0.8964   
          Std               0.0104               0.0104   
ARI       Mean              0.6310               0.6310   
          Std               0.0330               0.0330   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.8976  
          Std                         0.0104  
Precision Mean                        0.8996  
          Std                         0.0105  
Recall    Mean                        0.8976  
          Std                         0.0104  
F1 score  Mean                        0.8964  
          Std                         0.0104  
ARI       Mean                        0.6310  
          Std                         0.0330

### Mushroom

In [11]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('../Datasets/mushroom/agaricus-lepiota.data', header=None)
df.columns = ['cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape',
              'stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring',
              'veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat', 'class']

for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    
X = df.drop('class', axis=1).to_numpy()
y = df['class'].to_numpy()

In [5]:
compare_metrics_train_test(max_depth=3, X=X, y=y)

Finished: 1 iter.
Finished: 2 iter.
Finished: 3 iter.
Finished: 4 iter.
Finished: 5 iter.
Finished: 6 iter.
Finished: 7 iter.
Finished: 8 iter.
Finished: 9 iter.
Finished: 10 iter.
Finished: 11 iter.
Finished: 12 iter.
Finished: 13 iter.
Finished: 14 iter.
Finished: 15 iter.
Finished: 16 iter.
Finished: 17 iter.
Finished: 18 iter.
Finished: 19 iter.
Finished: 20 iter.
Finished: 21 iter.
Finished: 22 iter.
Finished: 23 iter.
Finished: 24 iter.
Finished: 25 iter.
Finished: 26 iter.
Finished: 27 iter.
Finished: 28 iter.
Finished: 29 iter.
Finished: 30 iter.
Finished: 31 iter.
Finished: 32 iter.
Finished: 33 iter.
Finished: 34 iter.
Finished: 35 iter.
Finished: 36 iter.
Finished: 37 iter.
Finished: 38 iter.
Finished: 39 iter.
Finished: 40 iter.
Finished: 41 iter.
Finished: 42 iter.
Finished: 43 iter.
Finished: 44 iter.
Finished: 45 iter.
Finished: 46 iter.
Finished: 47 iter.
Finished: 48 iter.
Finished: 49 iter.
Finished: 50 iter.

N, V, k, alpha, nmin, max_depth = (None, None, None, None,

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6008           0.6069         0.6012   
          Std        0.0112           0.0130         0.0110   
Precision Mean       0.5078           0.5392         0.5063   
          Std        0.0635           0.0418         0.0658   
Recall    Mean       0.6008           0.6069         0.6012   
          Std        0.0112           0.0130         0.0110   
F1 score  Mean       0.5262           0.5166         0.5254   
          Std        0.0221           0.0213         0.0229   
ARI       Mean       0.3096           0.2979         0.3232   
          Std        0.0170           0.0198         0.0198   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.5830   0.6181       0.6181   
          Std                           0.0133   0.0098       0.0098   
Precision Mean                          0.5275   0.4701       0.4701   
          Std                           0.0490   0.0124       0.0124   
Recall    Mean                          0.5830   0.6181       0.6181   
          Std                           0.0133   0.0098       0.0098   
F1 score  Mean                          0.5106   0.5240       0.5240   
          Std                           0.0230   0.0117       0.0117   
ARI       Mean                          0.2755   0.3593       0.3593   
          Std                           0.0306   0.0120       0.0120   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.5807               0.6003   
          Std               0.0093               0.0110   
Precision Mean              0.5154               0.5177   
          Std               0.0344               0.0690   
Recall    Mean              0.5807               0.6003   
          Std               0.0093               0.0110   
F1 score  Mean              0.5053               0.5292   
          Std               0.0237               0.0242   
ARI       Mean              0.2476               0.3076   
          Std               0.0277               0.0168   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.5970  
          Std                         0.0133  
Precision Mean                        0.5334  
          Std                         0.0627  
Recall    Mean                        0.5970  
          Std                         0.0133  
F1 score  Mean                        0.5177  
          Std                         0.0245  
ARI       Mean                        0.2894  
          Std                         0.0206

In [6]:
compare_metrics_train_test(max_depth=4, X=X, y=y)

Finished: 1 iter.
Finished: 2 iter.
Finished: 3 iter.
Finished: 4 iter.
Finished: 5 iter.
Finished: 6 iter.
Finished: 7 iter.
Finished: 8 iter.
Finished: 9 iter.
Finished: 10 iter.
Finished: 11 iter.
Finished: 12 iter.
Finished: 13 iter.
Finished: 14 iter.
Finished: 15 iter.
Finished: 16 iter.
Finished: 17 iter.
Finished: 18 iter.
Finished: 19 iter.
Finished: 20 iter.
Finished: 21 iter.
Finished: 22 iter.
Finished: 23 iter.
Finished: 24 iter.
Finished: 25 iter.
Finished: 26 iter.
Finished: 27 iter.
Finished: 28 iter.
Finished: 29 iter.
Finished: 30 iter.
Finished: 31 iter.
Finished: 32 iter.
Finished: 33 iter.
Finished: 34 iter.
Finished: 35 iter.
Finished: 36 iter.
Finished: 37 iter.
Finished: 38 iter.
Finished: 39 iter.
Finished: 40 iter.
Finished: 41 iter.
Finished: 42 iter.
Finished: 43 iter.
Finished: 44 iter.
Finished: 45 iter.
Finished: 46 iter.
Finished: 47 iter.
Finished: 48 iter.
Finished: 49 iter.
Finished: 50 iter.

N, V, k, alpha, nmin, max_depth = (None, None, None, None,

b = 1  entropy_sklearn  b = p_l ^ 0.5  \
Metric    Statistic                                           
Accuracy  Mean       0.6373           0.6403         0.6362   
          Std        0.0123           0.0103         0.0119   
Precision Mean       0.5943           0.6432         0.6030   
          Std        0.0415           0.0591         0.0410   
Recall    Mean       0.6373           0.6403         0.6362   
          Std        0.0123           0.0103         0.0119   
F1 score  Mean       0.5995           0.5940         0.6020   
          Std        0.0271           0.0298         0.0269   
ARI       Mean       0.3995           0.3921         0.3944   
          Std        0.0249           0.0290         0.0257   

                     b = (p_l*(1 - p_l)) ^ 0.5  b = p_l  b = p_l ^ 2  \
Metric    Statistic                                                    
Accuracy  Mean                          0.6078   0.6377       0.6455   
          Std                           0.0171   0.0093       0.0101   
Precision Mean                          0.6077   0.5951       0.5918   
          Std                           0.0501   0.0702       0.0694   
Recall    Mean                          0.6078   0.6377       0.6455   
          Std                           0.0171   0.0093       0.0101   
F1 score  Mean                          0.5418   0.5802       0.5846   
          Std                           0.0264   0.0268       0.0278   
ARI       Mean                          0.3254   0.3565       0.3712   
          Std                           0.0386   0.0120       0.0129   

                     b = -log(p_l)  b = -p_l * log(p_l)  \
Metric    Statistic                                       
Accuracy  Mean              0.6111               0.6368   
          Std               0.0114               0.0124   
Precision Mean              0.6431               0.5950   
          Std               0.0635               0.0395   
Recall    Mean              0.6111               0.6368   
          Std               0.0114               0.0124   
F1 score  Mean              0.5671               0.6000   
          Std               0.0310               0.0260   
ARI       Mean              0.3142               0.3991   
          Std               0.0251               0.0249   

                     b = -p_l^0.5 * log(p_l)  
Metric    Statistic                           
Accuracy  Mean                        0.6352  
          Std                         0.0174  
Precision Mean                        0.6332  
          Std                         0.0465  
Recall    Mean                        0.6352  
          Std                         0.0174  
F1 score  Mean                        0.5863  
          Std                         0.0340  
ARI       Mean                        0.3781  
          Std                         0.0311